<a href="https://colab.research.google.com/github/Salehin555/MOE-RUL-Prediction/blob/main/RUL_PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FD001

In [85]:
# ============================================================
# FINAL Interpretable MOE for C-MAPSS RUL (ONE-CELL Colab)
# Fixes:
#  - Noisy Top-K gating + load balancing + entropy bonus (Sub-Obj 1)
#  - Physics constraints: monotonic + smooth (Sub-Obj 2)
#  - UQ: NLL + MC Dropout + PI calibration (Sub-Obj 3)
#  - Transfer-ready structure (Sub-Obj 4 hooks; single-domain run here)
# ============================================================

# ---------------------------
# USER SETTINGS
# ---------------------------
DATA_DIR = "/content"
PRETRAIN_DOMAINS = ["FD001"]
TARGETS = ["FD001"]
DO_FINETUNE = False
ENSEMBLE_SIZE = 1

# ---------------------------
# Install deps
# ---------------------------
import sys, subprocess, importlib
def _ensure(pkg, import_name=None):
    name = import_name or pkg
    try:
        importlib.import_module(name)
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "-q", "install", pkg])

_ensure("scikit-learn", "sklearn")
_ensure("scipy", "scipy")

# ---------------------------
# Imports
# ---------------------------
import os, math, random
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Sampler

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import norm

# ---------------------------
# Config
# ---------------------------
@dataclass
class CFG:
    window: int = 30
    stride: int = 1
    max_rul: int = 125
    normalize_y: bool = True
    val_ratio_units: float = 0.2
    seed: int = 42

    drop_low_var: bool = True
    low_var_thresh: float = 1e-6

    n_experts: int = 4
    top_k: int = 2
    enc_hidden: int = 192
    head_hidden: int = 192
    dropout: float = 0.08
    gate_dropout: float = 0.05

    gate_noise_max: float = 1.0
    gate_noise_min: float = 0.15
    temp_max: float = 2.0
    temp_min: float = 0.7

    epochs: int = 85
    batch_size: int = 128
    block_len: int = 12
    lr: float = 2e-3
    weight_decay: float = 1e-4
    grad_clip: float = 1.0
    early_stop_patience: int = 14

    aux_mse_weight: float = 0.55
    huber_weight: float = 0.15
    huber_delta: float = 0.08  # in normalized y scale

    lambda_mono: float = 0.10
    lambda_smooth: float = 0.03
    lambda_lb: float = 0.25
    lambda_ent: float = 0.02
    lambda_div: float = 0.01
    lambda_dead: float = 0.06
    dead_floor: float = 0.03

    warmup_epochs: int = 12
    ramp_epochs: int = 20

    mc_samples: int = 30
    pi_alpha: float = 0.10  # 90% PI

cfg = CFG()

# ---------------------------
# Utils
# ---------------------------
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_seed(cfg.seed)

def ensure_exists(path: str):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Missing file: {path}")

def base_feature_columns() -> List[str]:
    return ["op1", "op2", "op3"] + [f"s{i}" for i in range(1, 22)]

def ramp(ep, warmup, ramp_len):
    if ep <= warmup: return 0.0
    return float(min(1.0, (ep - warmup) / max(1, ramp_len)))

def lerp(a, b, t):
    return a + (b - a) * t

# ---------------------------
# C-MAPSS load
# ---------------------------
def load_cmapss_split(data_dir: str, fd: str):
    train_file = os.path.join(data_dir, f"train_{fd}.txt")
    test_file  = os.path.join(data_dir, f"test_{fd}.txt")
    rul_file   = os.path.join(data_dir, f"RUL_{fd}.txt")
    ensure_exists(train_file); ensure_exists(test_file); ensure_exists(rul_file)

    train_df = pd.read_csv(train_file, sep=r"\s+", header=None)
    test_df  = pd.read_csv(test_file,  sep=r"\s+", header=None)
    rul_df   = pd.read_csv(rul_file,   sep=r"\s+", header=None)

    cols = ["unit","cycle","op1","op2","op3"] + [f"s{i}" for i in range(1,22)]
    train_df.columns = cols
    test_df.columns  = cols
    rul_df.columns   = ["RUL_last"]
    return train_df, test_df, rul_df

def add_rul_train(df: pd.DataFrame, max_rul: int):
    df = df.copy()
    max_cycle = df.groupby("unit")["cycle"].max()
    df["RUL"] = df.apply(lambda r: max_cycle.loc[r["unit"]] - r["cycle"], axis=1)
    df["RUL"] = df["RUL"].clip(upper=max_rul)
    return df

def add_rul_test(test_df: pd.DataFrame, rul_df: pd.DataFrame, max_rul: int):
    df = test_df.copy()
    max_cycle = df.groupby("unit")["cycle"].max()
    rul_last = rul_df["RUL_last"].values
    mapping = {u: rul_last[u-1] for u in sorted(df["unit"].unique())}
    df["RUL"] = df.apply(lambda r: (max_cycle.loc[r["unit"]] + mapping[r["unit"]]) - r["cycle"], axis=1)
    df["RUL"] = df["RUL"].clip(upper=max_rul)
    return df

def make_windows(df, window, stride, feature_cols, target_col="RUL"):
    xs, ys, units, cycles = [], [], [], []
    for unit, g in df.groupby("unit"):
        g = g.sort_values("cycle")
        feats = g[feature_cols].values.astype(np.float32)
        targ  = g[target_col].values.astype(np.float32)
        cyc   = g["cycle"].values.astype(np.int32)

        for end in range(window-1, len(g), stride):
            start = end-window+1
            xs.append(feats[start:end+1])
            ys.append(targ[end])
            units.append(unit)
            cycles.append(cyc[end])
    return np.stack(xs), np.array(ys), np.array(units), np.array(cycles)

def last_window_per_unit(X, y, unit_ids, cycles):
    idx=[]
    for u in np.unique(unit_ids):
        m = unit_ids==u
        i = np.argmax(cycles[m])
        idx.append(np.where(m)[0][i])
    idx=np.array(idx)
    return X[idx], y[idx], unit_ids[idx], cycles[idx]

def split_by_units(unit_ids, val_ratio, seed):
    rng=np.random.default_rng(seed)
    units=np.unique(unit_ids)
    rng.shuffle(units)
    n_val=max(1,int(len(units)*val_ratio))
    val=set(units[:n_val].tolist())
    tr=np.array([i for i,u in enumerate(unit_ids) if u not in val])
    va=np.array([i for i,u in enumerate(unit_ids) if u in val])
    return tr, va

# ---------------------------
# Dataset + contiguous sampler
# ---------------------------
class RULWindowDataset(Dataset):
    def __init__(self, X, y, unit_ids, cycles, domain_ids):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.unit_ids = np.array(unit_ids)
        self.cycles = np.array(cycles)
        self.domain_ids = torch.tensor(domain_ids, dtype=torch.long)

    def __len__(self): return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], int(self.unit_ids[idx]), int(self.cycles[idx]), self.domain_ids[idx]

class ContiguousEngineBatchSampler(Sampler):
    def __init__(self, unit_ids, cycles, batch_size, block_len, shuffle=True):
        self.unit_ids=np.array(unit_ids)
        self.cycles=np.array(cycles)
        self.batch_size=batch_size
        self.block_len=block_len
        self.shuffle=shuffle
        self.units=np.unique(self.unit_ids)
        self.unit_to_sorted={}
        for u in self.units:
            idx=np.where(self.unit_ids==u)[0]
            idx=idx[np.argsort(self.cycles[idx])]
            self.unit_to_sorted[u]=idx
        self.engines_per_batch=max(1,batch_size//block_len)

    def __iter__(self):
        units=self.units.copy()
        if self.shuffle: np.random.shuffle(units)
        batch=[]
        for u in units:
            idx=self.unit_to_sorted[u]
            L=len(idx)
            if L<=self.block_len:
                take=np.random.choice(idx,size=self.block_len,replace=True)
            else:
                s=np.random.randint(0,L-self.block_len)
                take=idx[s:s+self.block_len]
            batch.extend(take.tolist())
            if len(batch)>=self.engines_per_batch*self.block_len:
                yield batch[:self.batch_size]
                batch=[]
        if batch: yield batch

    def __len__(self):
        return math.ceil(len(self.units)/self.engines_per_batch)

# ---------------------------
# Physics losses
# ---------------------------
def physics_losses(pred_mean, unit_ids, cycles, margin=0.0):
    unit_ids=np.array(unit_ids); cycles=np.array(cycles)
    mono_terms=[]; smooth_terms=[]
    for u in np.unique(unit_ids):
        idx=np.where(unit_ids==u)[0]
        if len(idx)<2: continue
        ord_idx=idx[np.argsort(cycles[idx])]
        p=pred_mean[ord_idx]
        mono=F.relu(p[1:]-p[:-1]+margin)
        mono_terms.append(mono.mean())
        if len(ord_idx)>=3:
            second=p[2:]-2*p[1:-1]+p[:-2]
            smooth_terms.append(torch.abs(second).mean())
    mono_loss=torch.stack(mono_terms).mean() if mono_terms else pred_mean.new_tensor(0.)
    smooth_loss=torch.stack(smooth_terms).mean() if smooth_terms else pred_mean.new_tensor(0.)
    return mono_loss, smooth_loss

# ---------------------------
# Gating + load balance
# ---------------------------
class GatingNet(nn.Module):
    def __init__(self, in_dim, n_experts, gate_dropout):
        super().__init__()
        h=max(64,in_dim//2)
        self.net=nn.Sequential(
            nn.Linear(in_dim,h),
            nn.ReLU(),
            nn.Dropout(gate_dropout),
            nn.Linear(h,n_experts),
        )
    def forward(self,x): return self.net(x)

def sparse_topk_softmax(logits, k, temperature):
    logits = logits / max(1e-6, temperature)
    topk_vals, topk_idx = torch.topk(logits, k=k, dim=-1)
    masked = torch.full_like(logits, float("-inf"))
    masked.scatter_(dim=-1, index=topk_idx, src=topk_vals)
    w = F.softmax(masked, dim=-1)
    return w, topk_idx

def switch_load_balance_loss(w, top1_idx):
    B,E=w.shape
    importance=w.sum(dim=0)/(B+1e-8)
    load=torch.bincount(top1_idx,minlength=E).float().to(w.device)/(B+1e-8)
    return E*torch.sum(importance*load)

def dead_expert_penalty(w, floor):
    avg = w.mean(dim=0)
    return F.relu(floor - avg).mean()

# ---------------------------
# UQ + calibration
# ---------------------------
def gaussian_nll(y, mean, log_var):
    return 0.5*(torch.exp(-log_var)*(y-mean)**2 + log_var)

def prediction_interval(mean, var, alpha):
    z = norm.ppf(1-alpha/2)
    std=np.sqrt(np.maximum(var,1e-8))
    return mean - z*std, mean + z*std

def coverage(y, lo, hi):
    return float(np.mean((y>=lo)&(y<=hi)))

# ✅ FIXED predict_mc: safe unpacking for 7-value forward()
@torch.no_grad()
def predict_mc(model, X, n_mc, batch_size=256):
    model.train()
    dl=DataLoader(torch.tensor(X,dtype=torch.float32),batch_size=batch_size,shuffle=False)
    means_all=[]; vars_all=[]
    for xb in dl:
        xb=xb.to(DEVICE)
        mc_m=[]; mc_v=[]
        for _ in range(n_mc):
            out = model(xb, use_topk=True, noise_std=0.0, temperature=1.0, train=True)
            mean, log_var = out[0], out[1]
            mc_m.append(mean)
            mc_v.append(torch.exp(log_var))
        mc_m=torch.stack(mc_m,dim=0)
        mc_v=torch.stack(mc_v,dim=0)
        mean_pred=mc_m.mean(dim=0)
        epistemic=mc_m.var(dim=0,unbiased=False)
        aleatoric=mc_v.mean(dim=0)
        total=aleatoric+epistemic
        means_all.append(mean_pred.cpu().numpy())
        vars_all.append(total.cpu().numpy())
    return np.concatenate(means_all), np.concatenate(vars_all)

def calibrate_scale_min_width(model, X_val, y_val_raw, alpha, n_mc, max_over=0.01):
    mean, var = predict_mc(model, X_val, n_mc=n_mc)
    if cfg.normalize_y:
        mean = mean * cfg.max_rul
        var  = var  * (cfg.max_rul**2)
    std = np.sqrt(np.maximum(var,1e-8))
    z = norm.ppf(1-alpha/2)
    target = 1-alpha
    scales = np.linspace(0.6, 3.0, 121)
    best=None
    for s in scales:
        lo = mean - z*s*std
        hi = mean + z*s*std
        cov = np.mean((y_val_raw>=lo)&(y_val_raw<=hi))
        if cov >= (target - max_over):
            best=s
            break
    return float(best if best is not None else scales[-1])

# ---------------------------
# Model
# ---------------------------
class SharedEncoder(nn.Module):
    def __init__(self, n_features, hidden, dropout):
        super().__init__()
        self.conv1=nn.Conv1d(n_features,96,3,padding=1)
        self.conv2=nn.Conv1d(96,96,3,padding=1)
        self.bn1=nn.BatchNorm1d(96)
        self.bn2=nn.BatchNorm1d(96)
        self.gru=nn.GRU(96,hidden,batch_first=True)
        self.drop=nn.Dropout(dropout)
    def forward(self,x):
        x=x.transpose(1,2)
        x=F.relu(self.bn1(self.conv1(x)))
        x=F.relu(self.bn2(self.conv2(x)))
        x=x.transpose(1,2)
        _,h=self.gru(x)
        return self.drop(h[-1])

class ExpertHead(nn.Module):
    def __init__(self, in_dim, hidden, base_dropout, expert_id):
        super().__init__()
        width = hidden if expert_id%2==0 else max(64,hidden//2)
        drop  = min(0.30, base_dropout + 0.03*expert_id)
        self.emb = nn.Parameter(torch.randn(16)*0.02)
        self.net = nn.Sequential(
            nn.Linear(in_dim+16, width), nn.ReLU(), nn.Dropout(drop),
            nn.Linear(width, max(32,width//2)), nn.ReLU(), nn.Dropout(drop),
        )
        out=max(32,width//2)
        self.mu=nn.Linear(out,1)
        self.logv=nn.Linear(out,1)
    def forward(self,z):
        B=z.size(0)
        e=self.emb.unsqueeze(0).expand(B,-1)
        h=self.net(torch.cat([z,e],dim=-1))
        mu=self.mu(h).squeeze(-1)
        logv=self.logv(h).squeeze(-1).clamp(-9,4)
        return mu, logv

class InterpretableMoE(nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.enc = SharedEncoder(n_features,cfg.enc_hidden,cfg.dropout)
        self.experts = nn.ModuleList([ExpertHead(cfg.enc_hidden,cfg.head_hidden,cfg.dropout,i) for i in range(cfg.n_experts)])
        self.gate = GatingNet(cfg.enc_hidden+9, cfg.n_experts, cfg.gate_dropout)

    def forward(self, x, *, use_topk, noise_std, temperature, train):
        z = self.enc(x)
        op = x[:,:, :3]
        gate_in = torch.cat([z, op[:,-1,:], op.mean(dim=1), op.std(dim=1)], dim=-1)

        logits = self.gate(gate_in)
        if train and noise_std>0:
            logits = logits + torch.randn_like(logits)*noise_std

        if use_topk:
            w, topk_idx = sparse_topk_softmax(logits, cfg.top_k, temperature)
            top1 = topk_idx[:,0]
        else:
            w = F.softmax(logits/max(1e-6,temperature), dim=-1)
            top1 = torch.argmax(w, dim=-1)

        mus=[]; vars_=[]
        for ex in self.experts:
            mu, logv = ex(z)
            mus.append(mu); vars_.append(torch.exp(logv))
        mus=torch.stack(mus,dim=-1)
        vars_=torch.stack(vars_,dim=-1)

        mean=torch.sum(w*mus,dim=-1)
        second=torch.sum(w*(vars_+mus**2),dim=-1)
        var=(second-mean**2).clamp_min(1e-6)
        log_var=torch.log(var)

        lb = switch_load_balance_loss(w, top1)
        ent = -(w*torch.log(w+1e-8)).sum(dim=-1).mean()
        dead = dead_expert_penalty(w, cfg.dead_floor)

        div=0.0
        E=mus.shape[-1]
        for i in range(E):
            for j in range(i+1,E):
                a=mus[:,i]-mus[:,i].mean()
                b=mus[:,j]-mus[:,j].mean()
                div += torch.abs(F.cosine_similarity(a.unsqueeze(-1), b.unsqueeze(-1), dim=-1)).mean()
        div = div / max(1,(E*(E-1)//2))

        return mean, log_var, w, lb, ent, div, dead

# ---------------------------
# Prepare domain
# ---------------------------
def prepare_domain(fd, scaler=None, kept_cols=None):
    tr, te, rul = load_cmapss_split(DATA_DIR, fd)
    tr=add_rul_train(tr,cfg.max_rul)
    te=add_rul_test(te,rul,cfg.max_rul)

    all_cols = base_feature_columns()
    if kept_cols is None:
        kept_cols = all_cols
        if cfg.drop_low_var:
            v = tr[all_cols].var(axis=0).values
            kept_cols = [c for c,vv in zip(all_cols,v) if vv>cfg.low_var_thresh]
            for c in ["op1","op2","op3"]:
                if c not in kept_cols:
                    kept_cols = ["op1","op2","op3"] + [x for x in kept_cols if x not in ["op1","op2","op3"]]

    X_tr_all, y_tr_all, u_tr_all, c_tr_all = make_windows(tr, cfg.window, cfg.stride, kept_cols)
    X_te_all, y_te_all, u_te_all, c_te_all = make_windows(te, cfg.window, cfg.stride, kept_cols)
    X_te_last, y_te_last, u_te_last, c_te_last = last_window_per_unit(X_te_all, y_te_all, u_te_all, c_te_all)

    if scaler is None:
        scaler = StandardScaler()
        scaler.fit(X_tr_all.reshape(-1, X_tr_all.shape[-1]))

    X_tr_all = scaler.transform(X_tr_all.reshape(-1,X_tr_all.shape[-1])).reshape(X_tr_all.shape)
    X_te_last = scaler.transform(X_te_last.reshape(-1,X_te_last.shape[-1])).reshape(X_te_last.shape)

    tr_idx, va_idx = split_by_units(u_tr_all, cfg.val_ratio_units, cfg.seed)
    X_tr, y_tr, u_tr, c_tr = X_tr_all[tr_idx], y_tr_all[tr_idx], u_tr_all[tr_idx], c_tr_all[tr_idx]
    X_va, y_va, u_va, c_va = X_tr_all[va_idx], y_tr_all[va_idx], u_tr_all[va_idx], c_tr_all[va_idx]

    y_te_raw = y_te_last.copy()

    if cfg.normalize_y:
        y_tr = y_tr/cfg.max_rul
        y_va = y_va/cfg.max_rul
        y_te = y_te_last/cfg.max_rul
    else:
        y_te = y_te_last

    return {
        "scaler": scaler, "kept_cols": kept_cols,
        "train": (X_tr,y_tr,u_tr,c_tr),
        "val": (X_va,y_va,u_va,c_va),
        "test_last": (X_te_last,y_te,u_te_last,c_te_last),
        "test_last_y_raw": y_te_raw
    }

# ---------------------------
# Interpretability
# ---------------------------
@torch.no_grad()
def gate_stats(model, X, n_show=1024):
    model.eval()
    xb=torch.tensor(X[:n_show],dtype=torch.float32).to(DEVICE)
    out = model(xb, use_topk=True, noise_std=0.0, temperature=1.0, train=False)
    w = out[2].cpu().numpy()
    print("Gate usage (avg weights):", np.round(w.mean(axis=0),4))

# ---------------------------
# Training
# ---------------------------
def train_on_domain(domain):
    X_tr,y_tr,u_tr,c_tr = domain["train"]
    X_va,y_va,u_va,c_va = domain["val"]

    model = InterpretableMoE(n_features=X_tr.shape[-1]).to(DEVICE)

    ds_tr = RULWindowDataset(X_tr,y_tr,u_tr,c_tr,domain_ids=np.zeros(len(X_tr),dtype=int))
    ds_va = RULWindowDataset(X_va,y_va,u_va,c_va,domain_ids=np.zeros(len(X_va),dtype=int))

    sampler = ContiguousEngineBatchSampler(ds_tr.unit_ids, ds_tr.cycles, cfg.batch_size, cfg.block_len, shuffle=True)
    dl_tr = DataLoader(ds_tr, batch_sampler=sampler)
    dl_va = DataLoader(ds_va, batch_size=cfg.batch_size, shuffle=False)

    opt = torch.optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=cfg.epochs)

    best_val=float("inf"); best_state=None; bad=0

    for ep in range(1,cfg.epochs+1):
        s = ramp(ep, cfg.warmup_epochs, cfg.ramp_epochs)
        use_topk = (ep > cfg.warmup_epochs)
        noise_std = lerp(cfg.gate_noise_max, cfg.gate_noise_min, s)
        temperature = lerp(cfg.temp_max, cfg.temp_min, s)

        mono_w   = cfg.lambda_mono*s
        smooth_w = cfg.lambda_smooth*s
        lb_w     = cfg.lambda_lb*s
        ent_w    = cfg.lambda_ent*s
        div_w    = cfg.lambda_div*s
        dead_w   = cfg.lambda_dead*s

        model.train()
        tr_loss=0.0
        for xb,yb,ub,cb,db in dl_tr:
            xb=xb.to(DEVICE); yb=yb.to(DEVICE)
            opt.zero_grad(set_to_none=True)

            mean, logv, w, lb, ent, div, dead = model(
                xb, use_topk=use_topk, noise_std=noise_std, temperature=temperature, train=True
            )

            nll = gaussian_nll(yb, mean, logv).mean()
            mse = F.mse_loss(mean, yb)
            hub = F.huber_loss(mean, yb, delta=cfg.huber_delta)
            mono, smooth = physics_losses(mean, ub, cb)

            if ep <= cfg.warmup_epochs:
                loss = 1.0*mse + 0.10*nll + cfg.huber_weight*hub
            else:
                loss = nll + cfg.aux_mse_weight*mse + cfg.huber_weight*hub

            loss = loss + mono_w*mono + smooth_w*smooth + lb_w*lb - ent_w*ent + div_w*div + dead_w*dead

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.grad_clip)
            opt.step()
            tr_loss += float(loss.detach().cpu())

        sched.step()

        # val loss + quick metrics
        model.eval()
        va_loss=0.0
        with torch.no_grad():
            for xb,yb,ub,cb,db in dl_va:
                xb=xb.to(DEVICE); yb=yb.to(DEVICE)
                mean, logv, *_ = model(xb, use_topk=True, noise_std=0.0, temperature=1.0, train=False)
                nll = gaussian_nll(yb, mean, logv).mean()
                mse = F.mse_loss(mean, yb)
                hub = F.huber_loss(mean, yb, delta=cfg.huber_delta)
                va_loss += float((nll + cfg.aux_mse_weight*mse + cfg.huber_weight*hub).cpu())
        va_loss /= max(1,len(dl_va))

        yhat_val, _ = predict_mc(model, X_va, n_mc=1)
        if cfg.normalize_y:
            yhat_val = yhat_val*cfg.max_rul
            y_val_raw = y_va*cfg.max_rul
        else:
            y_val_raw = y_va
        r2 = r2_score(y_val_raw, yhat_val)
        rmse = math.sqrt(mean_squared_error(y_val_raw, yhat_val))
        print(f"[Epoch {ep:02d}] s={s:.2f} topk={int(use_topk)} val={va_loss:.4f} | R2={r2:.4f} RMSE={rmse:.3f}")

        if va_loss + 1e-6 < best_val:
            best_val=va_loss
            best_state={k:v.detach().cpu().clone() for k,v in model.state_dict().items()}
            bad=0
        else:
            bad+=1
            if bad>=cfg.early_stop_patience:
                print(f"Early stopping (best val={best_val:.4f})")
                break

    if best_state is not None:
        model.load_state_dict(best_state)
    return model

# ---------------------------
# RUN
# ---------------------------
for fd in set(PRETRAIN_DOMAINS + TARGETS):
    for f in [f"train_{fd}.txt", f"test_{fd}.txt", f"RUL_{fd}.txt"]:
        ensure_exists(os.path.join(DATA_DIR, f))

domains={}
scaler=None
kept=None
for fd in PRETRAIN_DOMAINS:
    domains[fd]=prepare_domain(fd, scaler=scaler, kept_cols=kept)
    scaler=domains[fd]["scaler"]
    kept=domains[fd]["kept_cols"]
for fd in TARGETS:
    if fd not in domains:
        domains[fd]=prepare_domain(fd, scaler=scaler, kept_cols=kept)

models=[]
for mi in range(ENSEMBLE_SIZE):
    print("\n==============================")
    print(f" Training model {mi+1}/{ENSEMBLE_SIZE}")
    print("==============================")
    set_seed(cfg.seed + mi)
    models.append(train_on_domain(domains[PRETRAIN_DOMAINS[0]]))

for fd in TARGETS:
    print("\n==============================")
    print(f" Target {fd} | LAST window per unit")
    print("==============================")

    X_te, y_te, *_ = domains[fd]["test_last"]
    y_te_raw = domains[fd]["test_last_y_raw"]
    X_va, y_va, *_ = domains[fd]["val"]
    y_va_raw = (y_va*cfg.max_rul) if cfg.normalize_y else y_va

    mean, var = predict_mc(models[0], X_te, n_mc=cfg.mc_samples)
    if cfg.normalize_y:
        mean_raw = mean*cfg.max_rul
        var_raw  = var*(cfg.max_rul**2)
    else:
        mean_raw = mean; var_raw = var

    scale = calibrate_scale_min_width(models[0], X_va, y_va_raw, cfg.pi_alpha, cfg.mc_samples, max_over=0.01)
    var_raw = (scale**2)*var_raw

    r2 = r2_score(y_te_raw, mean_raw)
    rmse = math.sqrt(mean_squared_error(y_te_raw, mean_raw))
    mae = mean_absolute_error(y_te_raw, mean_raw)

    lo, hi = prediction_interval(mean_raw, var_raw, cfg.pi_alpha)
    cov = coverage(y_te_raw, lo, hi)

    print(f"Point: R2={r2:.4f} RMSE={rmse:.3f} MAE={mae:.3f}")
    print(f"UQ: {int((1-cfg.pi_alpha)*100)}% PI coverage={cov:.3f} | mean width={(hi-lo).mean():.3f} | cal_scale={scale:.3f}")

    gate_stats(models[0], X_te)

print("\n✅ Done.")



 Training model 1/1
[Epoch 01] s=0.00 topk=0 val=0.1561 | R2=0.5270 RMSE=28.475
[Epoch 02] s=0.00 topk=0 val=-0.1708 | R2=0.6182 RMSE=25.583
[Epoch 03] s=0.00 topk=0 val=-0.3969 | R2=0.6275 RMSE=25.270
[Epoch 04] s=0.00 topk=0 val=-0.7160 | R2=0.7017 RMSE=22.614
[Epoch 05] s=0.00 topk=0 val=-0.8247 | R2=0.5785 RMSE=26.882
[Epoch 06] s=0.00 topk=0 val=-0.7446 | R2=0.4484 RMSE=30.750
[Epoch 07] s=0.00 topk=0 val=-1.2552 | R2=0.7263 RMSE=21.662
[Epoch 08] s=0.00 topk=0 val=-1.3979 | R2=0.7389 RMSE=21.156
[Epoch 09] s=0.00 topk=0 val=1.6415 | R2=-0.0068 RMSE=41.544
[Epoch 10] s=0.00 topk=0 val=-1.1568 | R2=0.6992 RMSE=22.709
[Epoch 11] s=0.00 topk=0 val=-1.0664 | R2=0.5933 RMSE=26.403
[Epoch 12] s=0.00 topk=0 val=-1.1788 | R2=0.7430 RMSE=20.990
[Epoch 13] s=0.05 topk=1 val=-1.4030 | R2=0.7206 RMSE=21.886
[Epoch 14] s=0.10 topk=1 val=-0.3117 | R2=-0.1598 RMSE=44.588
[Epoch 15] s=0.15 topk=1 val=-1.3494 | R2=0.7115 RMSE=22.237
[Epoch 16] s=0.20 topk=1 val=-1.3317 | R2=0.6758 RMSE=23.575
[Ep

FD002

In [95]:
# ============================================================
# FINAL Interpretable MOE for C-MAPSS RUL (ONE-CELL Colab)
# Fixes:
#  - Noisy Top-K gating + load balancing + entropy bonus (Sub-Obj 1)
#  - Physics constraints: monotonic + smooth (Sub-Obj 2)
#  - UQ: NLL + MC Dropout + PI calibration (Sub-Obj 3)
#  - Transfer-ready structure (Sub-Obj 4 hooks; single-domain run here)
# ============================================================

# ---------------------------
# USER SETTINGS
# ---------------------------
DATA_DIR = "/content"
PRETRAIN_DOMAINS = ["FD002"]
TARGETS = ["FD002"]
DO_FINETUNE = False
ENSEMBLE_SIZE = 1

# ---------------------------
# Install deps
# ---------------------------
import sys, subprocess, importlib
def _ensure(pkg, import_name=None):
    name = import_name or pkg
    try:
        importlib.import_module(name)
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "-q", "install", pkg])

_ensure("scikit-learn", "sklearn")
_ensure("scipy", "scipy")

# ---------------------------
# Imports
# ---------------------------
import os, math, random
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Sampler

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import norm

# ---------------------------
# Config
# ---------------------------
@dataclass
class CFG:
    window: int = 30
    stride: int = 1
    max_rul: int = 125
    normalize_y: bool = True
    val_ratio_units: float = 0.2
    seed: int = 42

    drop_low_var: bool = True
    low_var_thresh: float = 1e-6

    n_experts: int = 4
    top_k: int = 2
    enc_hidden: int = 192
    head_hidden: int = 192
    dropout: float = 0.08
    gate_dropout: float = 0.05

    gate_noise_max: float = 1.0
    gate_noise_min: float = 0.15
    temp_max: float = 2.0
    temp_min: float = 0.7

    epochs: int = 85
    batch_size: int = 128
    block_len: int = 12
    lr: float = 2e-3
    weight_decay: float = 1e-4
    grad_clip: float = 1.0
    early_stop_patience: int = 14

    aux_mse_weight: float = 0.55
    huber_weight: float = 0.15
    huber_delta: float = 0.08  # in normalized y scale

    lambda_mono: float = 0.10
    lambda_smooth: float = 0.03
    lambda_lb: float = 0.25
    lambda_ent: float = 0.02
    lambda_div: float = 0.01
    lambda_dead: float = 0.06
    dead_floor: float = 0.03

    warmup_epochs: int = 12
    ramp_epochs: int = 20

    mc_samples: int = 30
    pi_alpha: float = 0.10  # 90% PI

cfg = CFG()

# ---------------------------
# Utils
# ---------------------------
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_seed(cfg.seed)

def ensure_exists(path: str):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Missing file: {path}")

def base_feature_columns() -> List[str]:
    return ["op1", "op2", "op3"] + [f"s{i}" for i in range(1, 22)]

def ramp(ep, warmup, ramp_len):
    if ep <= warmup: return 0.0
    return float(min(1.0, (ep - warmup) / max(1, ramp_len)))

def lerp(a, b, t):
    return a + (b - a) * t

# ---------------------------
# C-MAPSS load
# ---------------------------
def load_cmapss_split(data_dir: str, fd: str):
    train_file = os.path.join(data_dir, f"train_{fd}.txt")
    test_file  = os.path.join(data_dir, f"test_{fd}.txt")
    rul_file   = os.path.join(data_dir, f"RUL_{fd}.txt")
    ensure_exists(train_file); ensure_exists(test_file); ensure_exists(rul_file)

    train_df = pd.read_csv(train_file, sep=r"\s+", header=None)
    test_df  = pd.read_csv(test_file,  sep=r"\s+", header=None)
    rul_df   = pd.read_csv(rul_file,   sep=r"\s+", header=None)

    cols = ["unit","cycle","op1","op2","op3"] + [f"s{i}" for i in range(1,22)]
    train_df.columns = cols
    test_df.columns  = cols
    rul_df.columns   = ["RUL_last"]
    return train_df, test_df, rul_df

def add_rul_train(df: pd.DataFrame, max_rul: int):
    df = df.copy()
    max_cycle = df.groupby("unit")["cycle"].max()
    df["RUL"] = df.apply(lambda r: max_cycle.loc[r["unit"]] - r["cycle"], axis=1)
    df["RUL"] = df["RUL"].clip(upper=max_rul)
    return df

def add_rul_test(test_df: pd.DataFrame, rul_df: pd.DataFrame, max_rul: int):
    df = test_df.copy()
    max_cycle = df.groupby("unit")["cycle"].max()
    rul_last = rul_df["RUL_last"].values
    mapping = {u: rul_last[u-1] for u in sorted(df["unit"].unique())}
    df["RUL"] = df.apply(lambda r: (max_cycle.loc[r["unit"]] + mapping[r["unit"]]) - r["cycle"], axis=1)
    df["RUL"] = df["RUL"].clip(upper=max_rul)
    return df

def make_windows(df, window, stride, feature_cols, target_col="RUL"):
    xs, ys, units, cycles = [], [], [], []
    for unit, g in df.groupby("unit"):
        g = g.sort_values("cycle")
        feats = g[feature_cols].values.astype(np.float32)
        targ  = g[target_col].values.astype(np.float32)
        cyc   = g["cycle"].values.astype(np.int32)

        for end in range(window-1, len(g), stride):
            start = end-window+1
            xs.append(feats[start:end+1])
            ys.append(targ[end])
            units.append(unit)
            cycles.append(cyc[end])
    return np.stack(xs), np.array(ys), np.array(units), np.array(cycles)

def last_window_per_unit(X, y, unit_ids, cycles):
    idx=[]
    for u in np.unique(unit_ids):
        m = unit_ids==u
        i = np.argmax(cycles[m])
        idx.append(np.where(m)[0][i])
    idx=np.array(idx)
    return X[idx], y[idx], unit_ids[idx], cycles[idx]

def split_by_units(unit_ids, val_ratio, seed):
    rng=np.random.default_rng(seed)
    units=np.unique(unit_ids)
    rng.shuffle(units)
    n_val=max(1,int(len(units)*val_ratio))
    val=set(units[:n_val].tolist())
    tr=np.array([i for i,u in enumerate(unit_ids) if u not in val])
    va=np.array([i for i,u in enumerate(unit_ids) if u in val])
    return tr, va

# ---------------------------
# Dataset + contiguous sampler
# ---------------------------
class RULWindowDataset(Dataset):
    def __init__(self, X, y, unit_ids, cycles, domain_ids):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.unit_ids = np.array(unit_ids)
        self.cycles = np.array(cycles)
        self.domain_ids = torch.tensor(domain_ids, dtype=torch.long)

    def __len__(self): return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], int(self.unit_ids[idx]), int(self.cycles[idx]), self.domain_ids[idx]

class ContiguousEngineBatchSampler(Sampler):
    def __init__(self, unit_ids, cycles, batch_size, block_len, shuffle=True):
        self.unit_ids=np.array(unit_ids)
        self.cycles=np.array(cycles)
        self.batch_size=batch_size
        self.block_len=block_len
        self.shuffle=shuffle
        self.units=np.unique(self.unit_ids)
        self.unit_to_sorted={}
        for u in self.units:
            idx=np.where(self.unit_ids==u)[0]
            idx=idx[np.argsort(self.cycles[idx])]
            self.unit_to_sorted[u]=idx
        self.engines_per_batch=max(1,batch_size//block_len)

    def __iter__(self):
        units=self.units.copy()
        if self.shuffle: np.random.shuffle(units)
        batch=[]
        for u in units:
            idx=self.unit_to_sorted[u]
            L=len(idx)
            if L<=self.block_len:
                take=np.random.choice(idx,size=self.block_len,replace=True)
            else:
                s=np.random.randint(0,L-self.block_len)
                take=idx[s:s+self.block_len]
            batch.extend(take.tolist())
            if len(batch)>=self.engines_per_batch*self.block_len:
                yield batch[:self.batch_size]
                batch=[]
        if batch: yield batch

    def __len__(self):
        return math.ceil(len(self.units)/self.engines_per_batch)

# ---------------------------
# Physics losses
# ---------------------------
def physics_losses(pred_mean, unit_ids, cycles, margin=0.0):
    unit_ids=np.array(unit_ids); cycles=np.array(cycles)
    mono_terms=[]; smooth_terms=[]
    for u in np.unique(unit_ids):
        idx=np.where(unit_ids==u)[0]
        if len(idx)<2: continue
        ord_idx=idx[np.argsort(cycles[idx])]
        p=pred_mean[ord_idx]
        mono=F.relu(p[1:]-p[:-1]+margin)
        mono_terms.append(mono.mean())
        if len(ord_idx)>=3:
            second=p[2:]-2*p[1:-1]+p[:-2]
            smooth_terms.append(torch.abs(second).mean())
    mono_loss=torch.stack(mono_terms).mean() if mono_terms else pred_mean.new_tensor(0.)
    smooth_loss=torch.stack(smooth_terms).mean() if smooth_terms else pred_mean.new_tensor(0.)
    return mono_loss, smooth_loss

# ---------------------------
# Gating + load balance
# ---------------------------
class GatingNet(nn.Module):
    def __init__(self, in_dim, n_experts, gate_dropout):
        super().__init__()
        h=max(64,in_dim//2)
        self.net=nn.Sequential(
            nn.Linear(in_dim,h),
            nn.ReLU(),
            nn.Dropout(gate_dropout),
            nn.Linear(h,n_experts),
        )
    def forward(self,x): return self.net(x)

def sparse_topk_softmax(logits, k, temperature):
    logits = logits / max(1e-6, temperature)
    topk_vals, topk_idx = torch.topk(logits, k=k, dim=-1)
    masked = torch.full_like(logits, float("-inf"))
    masked.scatter_(dim=-1, index=topk_idx, src=topk_vals)
    w = F.softmax(masked, dim=-1)
    return w, topk_idx

def switch_load_balance_loss(w, top1_idx):
    B,E=w.shape
    importance=w.sum(dim=0)/(B+1e-8)
    load=torch.bincount(top1_idx,minlength=E).float().to(w.device)/(B+1e-8)
    return E*torch.sum(importance*load)

def dead_expert_penalty(w, floor):
    avg = w.mean(dim=0)
    return F.relu(floor - avg).mean()

# ---------------------------
# UQ + calibration
# ---------------------------
def gaussian_nll(y, mean, log_var):
    return 0.5*(torch.exp(-log_var)*(y-mean)**2 + log_var)

def prediction_interval(mean, var, alpha):
    z = norm.ppf(1-alpha/2)
    std=np.sqrt(np.maximum(var,1e-8))
    return mean - z*std, mean + z*std

def coverage(y, lo, hi):
    return float(np.mean((y>=lo)&(y<=hi)))

# ✅ FIXED predict_mc: safe unpacking for 7-value forward()
@torch.no_grad()
def predict_mc(model, X, n_mc, batch_size=256):
    model.train()
    dl=DataLoader(torch.tensor(X,dtype=torch.float32),batch_size=batch_size,shuffle=False)
    means_all=[]; vars_all=[]
    for xb in dl:
        xb=xb.to(DEVICE)
        mc_m=[]; mc_v=[]
        for _ in range(n_mc):
            out = model(xb, use_topk=True, noise_std=0.0, temperature=1.0, train=True)
            mean, log_var = out[0], out[1]
            mc_m.append(mean)
            mc_v.append(torch.exp(log_var))
        mc_m=torch.stack(mc_m,dim=0)
        mc_v=torch.stack(mc_v,dim=0)
        mean_pred=mc_m.mean(dim=0)
        epistemic=mc_m.var(dim=0,unbiased=False)
        aleatoric=mc_v.mean(dim=0)
        total=aleatoric+epistemic
        means_all.append(mean_pred.cpu().numpy())
        vars_all.append(total.cpu().numpy())
    return np.concatenate(means_all), np.concatenate(vars_all)

def calibrate_scale_min_width(model, X_val, y_val_raw, alpha, n_mc, max_over=0.01):
    mean, var = predict_mc(model, X_val, n_mc=n_mc)
    if cfg.normalize_y:
        mean = mean * cfg.max_rul
        var  = var  * (cfg.max_rul**2)
    std = np.sqrt(np.maximum(var,1e-8))
    z = norm.ppf(1-alpha/2)
    target = 1-alpha
    scales = np.linspace(0.6, 3.0, 121)
    best=None
    for s in scales:
        lo = mean - z*s*std
        hi = mean + z*s*std
        cov = np.mean((y_val_raw>=lo)&(y_val_raw<=hi))
        if cov >= (target - max_over):
            best=s
            break
    return float(best if best is not None else scales[-1])

# ---------------------------
# Model
# ---------------------------
class SharedEncoder(nn.Module):
    def __init__(self, n_features, hidden, dropout):
        super().__init__()
        self.conv1=nn.Conv1d(n_features,96,3,padding=1)
        self.conv2=nn.Conv1d(96,96,3,padding=1)
        self.bn1=nn.BatchNorm1d(96)
        self.bn2=nn.BatchNorm1d(96)
        self.gru=nn.GRU(96,hidden,batch_first=True)
        self.drop=nn.Dropout(dropout)
    def forward(self,x):
        x=x.transpose(1,2)
        x=F.relu(self.bn1(self.conv1(x)))
        x=F.relu(self.bn2(self.conv2(x)))
        x=x.transpose(1,2)
        _,h=self.gru(x)
        return self.drop(h[-1])

class ExpertHead(nn.Module):
    def __init__(self, in_dim, hidden, base_dropout, expert_id):
        super().__init__()
        width = hidden if expert_id%2==0 else max(64,hidden//2)
        drop  = min(0.30, base_dropout + 0.03*expert_id)
        self.emb = nn.Parameter(torch.randn(16)*0.02)
        self.net = nn.Sequential(
            nn.Linear(in_dim+16, width), nn.ReLU(), nn.Dropout(drop),
            nn.Linear(width, max(32,width//2)), nn.ReLU(), nn.Dropout(drop),
        )
        out=max(32,width//2)
        self.mu=nn.Linear(out,1)
        self.logv=nn.Linear(out,1)
    def forward(self,z):
        B=z.size(0)
        e=self.emb.unsqueeze(0).expand(B,-1)
        h=self.net(torch.cat([z,e],dim=-1))
        mu=self.mu(h).squeeze(-1)
        logv=self.logv(h).squeeze(-1).clamp(-9,4)
        return mu, logv

class InterpretableMoE(nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.enc = SharedEncoder(n_features,cfg.enc_hidden,cfg.dropout)
        self.experts = nn.ModuleList([ExpertHead(cfg.enc_hidden,cfg.head_hidden,cfg.dropout,i) for i in range(cfg.n_experts)])
        self.gate = GatingNet(cfg.enc_hidden+9, cfg.n_experts, cfg.gate_dropout)

    def forward(self, x, *, use_topk, noise_std, temperature, train):
        z = self.enc(x)
        op = x[:,:, :3]
        gate_in = torch.cat([z, op[:,-1,:], op.mean(dim=1), op.std(dim=1)], dim=-1)

        logits = self.gate(gate_in)
        if train and noise_std>0:
            logits = logits + torch.randn_like(logits)*noise_std

        if use_topk:
            w, topk_idx = sparse_topk_softmax(logits, cfg.top_k, temperature)
            top1 = topk_idx[:,0]
        else:
            w = F.softmax(logits/max(1e-6,temperature), dim=-1)
            top1 = torch.argmax(w, dim=-1)

        mus=[]; vars_=[]
        for ex in self.experts:
            mu, logv = ex(z)
            mus.append(mu); vars_.append(torch.exp(logv))
        mus=torch.stack(mus,dim=-1)
        vars_=torch.stack(vars_,dim=-1)

        mean=torch.sum(w*mus,dim=-1)
        second=torch.sum(w*(vars_+mus**2),dim=-1)
        var=(second-mean**2).clamp_min(1e-6)
        log_var=torch.log(var)

        lb = switch_load_balance_loss(w, top1)
        ent = -(w*torch.log(w+1e-8)).sum(dim=-1).mean()
        dead = dead_expert_penalty(w, cfg.dead_floor)

        div=0.0
        E=mus.shape[-1]
        for i in range(E):
            for j in range(i+1,E):
                a=mus[:,i]-mus[:,i].mean()
                b=mus[:,j]-mus[:,j].mean()
                div += torch.abs(F.cosine_similarity(a.unsqueeze(-1), b.unsqueeze(-1), dim=-1)).mean()
        div = div / max(1,(E*(E-1)//2))

        return mean, log_var, w, lb, ent, div, dead

# ---------------------------
# Prepare domain
# ---------------------------
def prepare_domain(fd, scaler=None, kept_cols=None):
    tr, te, rul = load_cmapss_split(DATA_DIR, fd)
    tr=add_rul_train(tr,cfg.max_rul)
    te=add_rul_test(te,rul,cfg.max_rul)

    all_cols = base_feature_columns()
    if kept_cols is None:
        kept_cols = all_cols
        if cfg.drop_low_var:
            v = tr[all_cols].var(axis=0).values
            kept_cols = [c for c,vv in zip(all_cols,v) if vv>cfg.low_var_thresh]
            for c in ["op1","op2","op3"]:
                if c not in kept_cols:
                    kept_cols = ["op1","op2","op3"] + [x for x in kept_cols if x not in ["op1","op2","op3"]]

    X_tr_all, y_tr_all, u_tr_all, c_tr_all = make_windows(tr, cfg.window, cfg.stride, kept_cols)
    X_te_all, y_te_all, u_te_all, c_te_all = make_windows(te, cfg.window, cfg.stride, kept_cols)
    X_te_last, y_te_last, u_te_last, c_te_last = last_window_per_unit(X_te_all, y_te_all, u_te_all, c_te_all)

    if scaler is None:
        scaler = StandardScaler()
        scaler.fit(X_tr_all.reshape(-1, X_tr_all.shape[-1]))

    X_tr_all = scaler.transform(X_tr_all.reshape(-1,X_tr_all.shape[-1])).reshape(X_tr_all.shape)
    X_te_last = scaler.transform(X_te_last.reshape(-1,X_te_last.shape[-1])).reshape(X_te_last.shape)

    tr_idx, va_idx = split_by_units(u_tr_all, cfg.val_ratio_units, cfg.seed)
    X_tr, y_tr, u_tr, c_tr = X_tr_all[tr_idx], y_tr_all[tr_idx], u_tr_all[tr_idx], c_tr_all[tr_idx]
    X_va, y_va, u_va, c_va = X_tr_all[va_idx], y_tr_all[va_idx], u_tr_all[va_idx], c_tr_all[va_idx]

    y_te_raw = y_te_last.copy()

    if cfg.normalize_y:
        y_tr = y_tr/cfg.max_rul
        y_va = y_va/cfg.max_rul
        y_te = y_te_last/cfg.max_rul
    else:
        y_te = y_te_last

    return {
        "scaler": scaler, "kept_cols": kept_cols,
        "train": (X_tr,y_tr,u_tr,c_tr),
        "val": (X_va,y_va,u_va,c_va),
        "test_last": (X_te_last,y_te,u_te_last,c_te_last),
        "test_last_y_raw": y_te_raw
    }

# ---------------------------
# Interpretability
# ---------------------------
@torch.no_grad()
def gate_stats(model, X, n_show=1024):
    model.eval()
    xb=torch.tensor(X[:n_show],dtype=torch.float32).to(DEVICE)
    out = model(xb, use_topk=True, noise_std=0.0, temperature=1.0, train=False)
    w = out[2].cpu().numpy()
    print("Gate usage (avg weights):", np.round(w.mean(axis=0),4))

# ---------------------------
# Training
# ---------------------------
def train_on_domain(domain):
    X_tr,y_tr,u_tr,c_tr = domain["train"]
    X_va,y_va,u_va,c_va = domain["val"]

    model = InterpretableMoE(n_features=X_tr.shape[-1]).to(DEVICE)

    ds_tr = RULWindowDataset(X_tr,y_tr,u_tr,c_tr,domain_ids=np.zeros(len(X_tr),dtype=int))
    ds_va = RULWindowDataset(X_va,y_va,u_va,c_va,domain_ids=np.zeros(len(X_va),dtype=int))

    sampler = ContiguousEngineBatchSampler(ds_tr.unit_ids, ds_tr.cycles, cfg.batch_size, cfg.block_len, shuffle=True)
    dl_tr = DataLoader(ds_tr, batch_sampler=sampler)
    dl_va = DataLoader(ds_va, batch_size=cfg.batch_size, shuffle=False)

    opt = torch.optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=cfg.epochs)

    best_val=float("inf"); best_state=None; bad=0

    for ep in range(1,cfg.epochs+1):
        s = ramp(ep, cfg.warmup_epochs, cfg.ramp_epochs)
        use_topk = (ep > cfg.warmup_epochs)
        noise_std = lerp(cfg.gate_noise_max, cfg.gate_noise_min, s)
        temperature = lerp(cfg.temp_max, cfg.temp_min, s)

        mono_w   = cfg.lambda_mono*s
        smooth_w = cfg.lambda_smooth*s
        lb_w     = cfg.lambda_lb*s
        ent_w    = cfg.lambda_ent*s
        div_w    = cfg.lambda_div*s
        dead_w   = cfg.lambda_dead*s

        model.train()
        tr_loss=0.0
        for xb,yb,ub,cb,db in dl_tr:
            xb=xb.to(DEVICE); yb=yb.to(DEVICE)
            opt.zero_grad(set_to_none=True)

            mean, logv, w, lb, ent, div, dead = model(
                xb, use_topk=use_topk, noise_std=noise_std, temperature=temperature, train=True
            )

            nll = gaussian_nll(yb, mean, logv).mean()
            mse = F.mse_loss(mean, yb)
            hub = F.huber_loss(mean, yb, delta=cfg.huber_delta)
            mono, smooth = physics_losses(mean, ub, cb)

            if ep <= cfg.warmup_epochs:
                loss = 1.0*mse + 0.10*nll + cfg.huber_weight*hub
            else:
                loss = nll + cfg.aux_mse_weight*mse + cfg.huber_weight*hub

            loss = loss + mono_w*mono + smooth_w*smooth + lb_w*lb - ent_w*ent + div_w*div + dead_w*dead

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.grad_clip)
            opt.step()
            tr_loss += float(loss.detach().cpu())

        sched.step()

        # val loss + quick metrics
        model.eval()
        va_loss=0.0
        with torch.no_grad():
            for xb,yb,ub,cb,db in dl_va:
                xb=xb.to(DEVICE); yb=yb.to(DEVICE)
                mean, logv, *_ = model(xb, use_topk=True, noise_std=0.0, temperature=1.0, train=False)
                nll = gaussian_nll(yb, mean, logv).mean()
                mse = F.mse_loss(mean, yb)
                hub = F.huber_loss(mean, yb, delta=cfg.huber_delta)
                va_loss += float((nll + cfg.aux_mse_weight*mse + cfg.huber_weight*hub).cpu())
        va_loss /= max(1,len(dl_va))

        yhat_val, _ = predict_mc(model, X_va, n_mc=1)
        if cfg.normalize_y:
            yhat_val = yhat_val*cfg.max_rul
            y_val_raw = y_va*cfg.max_rul
        else:
            y_val_raw = y_va
        r2 = r2_score(y_val_raw, yhat_val)
        rmse = math.sqrt(mean_squared_error(y_val_raw, yhat_val))
        print(f"[Epoch {ep:02d}] s={s:.2f} topk={int(use_topk)} val={va_loss:.4f} | R2={r2:.4f} RMSE={rmse:.3f}")

        if va_loss + 1e-6 < best_val:
            best_val=va_loss
            best_state={k:v.detach().cpu().clone() for k,v in model.state_dict().items()}
            bad=0
        else:
            bad+=1
            if bad>=cfg.early_stop_patience:
                print(f"Early stopping (best val={best_val:.4f})")
                break

    if best_state is not None:
        model.load_state_dict(best_state)
    return model

# ---------------------------
# RUN
# ---------------------------
for fd in set(PRETRAIN_DOMAINS + TARGETS):
    for f in [f"train_{fd}.txt", f"test_{fd}.txt", f"RUL_{fd}.txt"]:
        ensure_exists(os.path.join(DATA_DIR, f))

domains={}
scaler=None
kept=None
for fd in PRETRAIN_DOMAINS:
    domains[fd]=prepare_domain(fd, scaler=scaler, kept_cols=kept)
    scaler=domains[fd]["scaler"]
    kept=domains[fd]["kept_cols"]
for fd in TARGETS:
    if fd not in domains:
        domains[fd]=prepare_domain(fd, scaler=scaler, kept_cols=kept)

models=[]
for mi in range(ENSEMBLE_SIZE):
    print("\n==============================")
    print(f" Training model {mi+1}/{ENSEMBLE_SIZE}")
    print("==============================")
    set_seed(cfg.seed + mi)
    models.append(train_on_domain(domains[PRETRAIN_DOMAINS[0]]))

for fd in TARGETS:
    print("\n==============================")
    print(f" Target {fd} | LAST window per unit")
    print("==============================")

    X_te, y_te, *_ = domains[fd]["test_last"]
    y_te_raw = domains[fd]["test_last_y_raw"]
    X_va, y_va, *_ = domains[fd]["val"]
    y_va_raw = (y_va*cfg.max_rul) if cfg.normalize_y else y_va

    mean, var = predict_mc(models[0], X_te, n_mc=cfg.mc_samples)
    if cfg.normalize_y:
        mean_raw = mean*cfg.max_rul
        var_raw  = var*(cfg.max_rul**2)
    else:
        mean_raw = mean; var_raw = var

    scale = calibrate_scale_min_width(models[0], X_va, y_va_raw, cfg.pi_alpha, cfg.mc_samples, max_over=0.01)
    var_raw = (scale**2)*var_raw

    r2 = r2_score(y_te_raw, mean_raw)
    rmse = math.sqrt(mean_squared_error(y_te_raw, mean_raw))
    mae = mean_absolute_error(y_te_raw, mean_raw)

    lo, hi = prediction_interval(mean_raw, var_raw, cfg.pi_alpha)
    cov = coverage(y_te_raw, lo, hi)

    print(f"Point: R2={r2:.4f} RMSE={rmse:.3f} MAE={mae:.3f}")
    print(f"UQ: {int((1-cfg.pi_alpha)*100)}% PI coverage={cov:.3f} | mean width={(hi-lo).mean():.3f} | cal_scale={scale:.3f}")

    gate_stats(models[0], X_te)

print("\n✅ Done.")



 Training model 1/1
[Epoch 01] s=0.00 topk=0 val=-0.2407 | R2=-0.0040 RMSE=41.841
[Epoch 02] s=0.00 topk=0 val=-0.5685 | R2=0.0405 RMSE=40.904
[Epoch 03] s=0.00 topk=0 val=-0.8230 | R2=0.5002 RMSE=29.521
[Epoch 04] s=0.00 topk=0 val=-0.8968 | R2=0.5992 RMSE=26.435
[Epoch 05] s=0.00 topk=0 val=-0.8805 | R2=0.5406 RMSE=28.303
[Epoch 06] s=0.00 topk=0 val=-0.9473 | R2=0.5696 RMSE=27.395
[Epoch 07] s=0.00 topk=0 val=-1.1210 | R2=0.6504 RMSE=24.692
[Epoch 08] s=0.00 topk=0 val=-1.1902 | R2=0.7201 RMSE=22.093
[Epoch 09] s=0.00 topk=0 val=-0.9997 | R2=0.4966 RMSE=29.627
[Epoch 10] s=0.00 topk=0 val=-1.1631 | R2=0.6726 RMSE=23.893
[Epoch 11] s=0.00 topk=0 val=-0.8876 | R2=0.4254 RMSE=31.653
[Epoch 12] s=0.00 topk=0 val=-0.9271 | R2=0.4042 RMSE=32.233
[Epoch 13] s=0.05 topk=1 val=-0.8902 | R2=0.6009 RMSE=26.379
[Epoch 14] s=0.10 topk=1 val=-0.5136 | R2=0.7117 RMSE=22.419
[Epoch 15] s=0.15 topk=1 val=-1.1794 | R2=0.6934 RMSE=23.123
[Epoch 16] s=0.20 topk=1 val=-1.1896 | R2=0.7095 RMSE=22.508
[E

FD003

In [96]:
# ============================================================
# FINAL Interpretable MOE for C-MAPSS RUL (ONE-CELL Colab)
# Fixes:
#  - Noisy Top-K gating + load balancing + entropy bonus (Sub-Obj 1)
#  - Physics constraints: monotonic + smooth (Sub-Obj 2)
#  - UQ: NLL + MC Dropout + PI calibration (Sub-Obj 3)
#  - Transfer-ready structure (Sub-Obj 4 hooks; single-domain run here)
# ============================================================

# ---------------------------
# USER SETTINGS
# ---------------------------
DATA_DIR = "/content"
PRETRAIN_DOMAINS = ["FD003"]
TARGETS = ["FD003"]
DO_FINETUNE = False
ENSEMBLE_SIZE = 1

# ---------------------------
# Install deps
# ---------------------------
import sys, subprocess, importlib
def _ensure(pkg, import_name=None):
    name = import_name or pkg
    try:
        importlib.import_module(name)
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "-q", "install", pkg])

_ensure("scikit-learn", "sklearn")
_ensure("scipy", "scipy")

# ---------------------------
# Imports
# ---------------------------
import os, math, random
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Sampler

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import norm

# ---------------------------
# Config
# ---------------------------
@dataclass
class CFG:
    window: int = 30
    stride: int = 1
    max_rul: int = 125
    normalize_y: bool = True
    val_ratio_units: float = 0.2
    seed: int = 42

    drop_low_var: bool = True
    low_var_thresh: float = 1e-6

    n_experts: int = 4
    top_k: int = 2
    enc_hidden: int = 192
    head_hidden: int = 192
    dropout: float = 0.08
    gate_dropout: float = 0.05

    gate_noise_max: float = 1.0
    gate_noise_min: float = 0.15
    temp_max: float = 2.0
    temp_min: float = 0.7

    epochs: int = 85
    batch_size: int = 128
    block_len: int = 12
    lr: float = 2e-3
    weight_decay: float = 1e-4
    grad_clip: float = 1.0
    early_stop_patience: int = 14

    aux_mse_weight: float = 0.55
    huber_weight: float = 0.15
    huber_delta: float = 0.08  # in normalized y scale

    lambda_mono: float = 0.10
    lambda_smooth: float = 0.03
    lambda_lb: float = 0.25
    lambda_ent: float = 0.02
    lambda_div: float = 0.01
    lambda_dead: float = 0.06
    dead_floor: float = 0.03

    warmup_epochs: int = 12
    ramp_epochs: int = 20

    mc_samples: int = 30
    pi_alpha: float = 0.10  # 90% PI

cfg = CFG()

# ---------------------------
# Utils
# ---------------------------
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_seed(cfg.seed)

def ensure_exists(path: str):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Missing file: {path}")

def base_feature_columns() -> List[str]:
    return ["op1", "op2", "op3"] + [f"s{i}" for i in range(1, 22)]

def ramp(ep, warmup, ramp_len):
    if ep <= warmup: return 0.0
    return float(min(1.0, (ep - warmup) / max(1, ramp_len)))

def lerp(a, b, t):
    return a + (b - a) * t

# ---------------------------
# C-MAPSS load
# ---------------------------
def load_cmapss_split(data_dir: str, fd: str):
    train_file = os.path.join(data_dir, f"train_{fd}.txt")
    test_file  = os.path.join(data_dir, f"test_{fd}.txt")
    rul_file   = os.path.join(data_dir, f"RUL_{fd}.txt")
    ensure_exists(train_file); ensure_exists(test_file); ensure_exists(rul_file)

    train_df = pd.read_csv(train_file, sep=r"\s+", header=None)
    test_df  = pd.read_csv(test_file,  sep=r"\s+", header=None)
    rul_df   = pd.read_csv(rul_file,   sep=r"\s+", header=None)

    cols = ["unit","cycle","op1","op2","op3"] + [f"s{i}" for i in range(1,22)]
    train_df.columns = cols
    test_df.columns  = cols
    rul_df.columns   = ["RUL_last"]
    return train_df, test_df, rul_df

def add_rul_train(df: pd.DataFrame, max_rul: int):
    df = df.copy()
    max_cycle = df.groupby("unit")["cycle"].max()
    df["RUL"] = df.apply(lambda r: max_cycle.loc[r["unit"]] - r["cycle"], axis=1)
    df["RUL"] = df["RUL"].clip(upper=max_rul)
    return df

def add_rul_test(test_df: pd.DataFrame, rul_df: pd.DataFrame, max_rul: int):
    df = test_df.copy()
    max_cycle = df.groupby("unit")["cycle"].max()
    rul_last = rul_df["RUL_last"].values
    mapping = {u: rul_last[u-1] for u in sorted(df["unit"].unique())}
    df["RUL"] = df.apply(lambda r: (max_cycle.loc[r["unit"]] + mapping[r["unit"]]) - r["cycle"], axis=1)
    df["RUL"] = df["RUL"].clip(upper=max_rul)
    return df

def make_windows(df, window, stride, feature_cols, target_col="RUL"):
    xs, ys, units, cycles = [], [], [], []
    for unit, g in df.groupby("unit"):
        g = g.sort_values("cycle")
        feats = g[feature_cols].values.astype(np.float32)
        targ  = g[target_col].values.astype(np.float32)
        cyc   = g["cycle"].values.astype(np.int32)

        for end in range(window-1, len(g), stride):
            start = end-window+1
            xs.append(feats[start:end+1])
            ys.append(targ[end])
            units.append(unit)
            cycles.append(cyc[end])
    return np.stack(xs), np.array(ys), np.array(units), np.array(cycles)

def last_window_per_unit(X, y, unit_ids, cycles):
    idx=[]
    for u in np.unique(unit_ids):
        m = unit_ids==u
        i = np.argmax(cycles[m])
        idx.append(np.where(m)[0][i])
    idx=np.array(idx)
    return X[idx], y[idx], unit_ids[idx], cycles[idx]

def split_by_units(unit_ids, val_ratio, seed):
    rng=np.random.default_rng(seed)
    units=np.unique(unit_ids)
    rng.shuffle(units)
    n_val=max(1,int(len(units)*val_ratio))
    val=set(units[:n_val].tolist())
    tr=np.array([i for i,u in enumerate(unit_ids) if u not in val])
    va=np.array([i for i,u in enumerate(unit_ids) if u in val])
    return tr, va

# ---------------------------
# Dataset + contiguous sampler
# ---------------------------
class RULWindowDataset(Dataset):
    def __init__(self, X, y, unit_ids, cycles, domain_ids):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.unit_ids = np.array(unit_ids)
        self.cycles = np.array(cycles)
        self.domain_ids = torch.tensor(domain_ids, dtype=torch.long)

    def __len__(self): return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], int(self.unit_ids[idx]), int(self.cycles[idx]), self.domain_ids[idx]

class ContiguousEngineBatchSampler(Sampler):
    def __init__(self, unit_ids, cycles, batch_size, block_len, shuffle=True):
        self.unit_ids=np.array(unit_ids)
        self.cycles=np.array(cycles)
        self.batch_size=batch_size
        self.block_len=block_len
        self.shuffle=shuffle
        self.units=np.unique(self.unit_ids)
        self.unit_to_sorted={}
        for u in self.units:
            idx=np.where(self.unit_ids==u)[0]
            idx=idx[np.argsort(self.cycles[idx])]
            self.unit_to_sorted[u]=idx
        self.engines_per_batch=max(1,batch_size//block_len)

    def __iter__(self):
        units=self.units.copy()
        if self.shuffle: np.random.shuffle(units)
        batch=[]
        for u in units:
            idx=self.unit_to_sorted[u]
            L=len(idx)
            if L<=self.block_len:
                take=np.random.choice(idx,size=self.block_len,replace=True)
            else:
                s=np.random.randint(0,L-self.block_len)
                take=idx[s:s+self.block_len]
            batch.extend(take.tolist())
            if len(batch)>=self.engines_per_batch*self.block_len:
                yield batch[:self.batch_size]
                batch=[]
        if batch: yield batch

    def __len__(self):
        return math.ceil(len(self.units)/self.engines_per_batch)

# ---------------------------
# Physics losses
# ---------------------------
def physics_losses(pred_mean, unit_ids, cycles, margin=0.0):
    unit_ids=np.array(unit_ids); cycles=np.array(cycles)
    mono_terms=[]; smooth_terms=[]
    for u in np.unique(unit_ids):
        idx=np.where(unit_ids==u)[0]
        if len(idx)<2: continue
        ord_idx=idx[np.argsort(cycles[idx])]
        p=pred_mean[ord_idx]
        mono=F.relu(p[1:]-p[:-1]+margin)
        mono_terms.append(mono.mean())
        if len(ord_idx)>=3:
            second=p[2:]-2*p[1:-1]+p[:-2]
            smooth_terms.append(torch.abs(second).mean())
    mono_loss=torch.stack(mono_terms).mean() if mono_terms else pred_mean.new_tensor(0.)
    smooth_loss=torch.stack(smooth_terms).mean() if smooth_terms else pred_mean.new_tensor(0.)
    return mono_loss, smooth_loss

# ---------------------------
# Gating + load balance
# ---------------------------
class GatingNet(nn.Module):
    def __init__(self, in_dim, n_experts, gate_dropout):
        super().__init__()
        h=max(64,in_dim//2)
        self.net=nn.Sequential(
            nn.Linear(in_dim,h),
            nn.ReLU(),
            nn.Dropout(gate_dropout),
            nn.Linear(h,n_experts),
        )
    def forward(self,x): return self.net(x)

def sparse_topk_softmax(logits, k, temperature):
    logits = logits / max(1e-6, temperature)
    topk_vals, topk_idx = torch.topk(logits, k=k, dim=-1)
    masked = torch.full_like(logits, float("-inf"))
    masked.scatter_(dim=-1, index=topk_idx, src=topk_vals)
    w = F.softmax(masked, dim=-1)
    return w, topk_idx

def switch_load_balance_loss(w, top1_idx):
    B,E=w.shape
    importance=w.sum(dim=0)/(B+1e-8)
    load=torch.bincount(top1_idx,minlength=E).float().to(w.device)/(B+1e-8)
    return E*torch.sum(importance*load)

def dead_expert_penalty(w, floor):
    avg = w.mean(dim=0)
    return F.relu(floor - avg).mean()

# ---------------------------
# UQ + calibration
# ---------------------------
def gaussian_nll(y, mean, log_var):
    return 0.5*(torch.exp(-log_var)*(y-mean)**2 + log_var)

def prediction_interval(mean, var, alpha):
    z = norm.ppf(1-alpha/2)
    std=np.sqrt(np.maximum(var,1e-8))
    return mean - z*std, mean + z*std

def coverage(y, lo, hi):
    return float(np.mean((y>=lo)&(y<=hi)))

# ✅ FIXED predict_mc: safe unpacking for 7-value forward()
@torch.no_grad()
def predict_mc(model, X, n_mc, batch_size=256):
    model.train()
    dl=DataLoader(torch.tensor(X,dtype=torch.float32),batch_size=batch_size,shuffle=False)
    means_all=[]; vars_all=[]
    for xb in dl:
        xb=xb.to(DEVICE)
        mc_m=[]; mc_v=[]
        for _ in range(n_mc):
            out = model(xb, use_topk=True, noise_std=0.0, temperature=1.0, train=True)
            mean, log_var = out[0], out[1]
            mc_m.append(mean)
            mc_v.append(torch.exp(log_var))
        mc_m=torch.stack(mc_m,dim=0)
        mc_v=torch.stack(mc_v,dim=0)
        mean_pred=mc_m.mean(dim=0)
        epistemic=mc_m.var(dim=0,unbiased=False)
        aleatoric=mc_v.mean(dim=0)
        total=aleatoric+epistemic
        means_all.append(mean_pred.cpu().numpy())
        vars_all.append(total.cpu().numpy())
    return np.concatenate(means_all), np.concatenate(vars_all)

def calibrate_scale_min_width(model, X_val, y_val_raw, alpha, n_mc, max_over=0.01):
    mean, var = predict_mc(model, X_val, n_mc=n_mc)
    if cfg.normalize_y:
        mean = mean * cfg.max_rul
        var  = var  * (cfg.max_rul**2)
    std = np.sqrt(np.maximum(var,1e-8))
    z = norm.ppf(1-alpha/2)
    target = 1-alpha
    scales = np.linspace(0.6, 3.0, 121)
    best=None
    for s in scales:
        lo = mean - z*s*std
        hi = mean + z*s*std
        cov = np.mean((y_val_raw>=lo)&(y_val_raw<=hi))
        if cov >= (target - max_over):
            best=s
            break
    return float(best if best is not None else scales[-1])

# ---------------------------
# Model
# ---------------------------
class SharedEncoder(nn.Module):
    def __init__(self, n_features, hidden, dropout):
        super().__init__()
        self.conv1=nn.Conv1d(n_features,96,3,padding=1)
        self.conv2=nn.Conv1d(96,96,3,padding=1)
        self.bn1=nn.BatchNorm1d(96)
        self.bn2=nn.BatchNorm1d(96)
        self.gru=nn.GRU(96,hidden,batch_first=True)
        self.drop=nn.Dropout(dropout)
    def forward(self,x):
        x=x.transpose(1,2)
        x=F.relu(self.bn1(self.conv1(x)))
        x=F.relu(self.bn2(self.conv2(x)))
        x=x.transpose(1,2)
        _,h=self.gru(x)
        return self.drop(h[-1])

class ExpertHead(nn.Module):
    def __init__(self, in_dim, hidden, base_dropout, expert_id):
        super().__init__()
        width = hidden if expert_id%2==0 else max(64,hidden//2)
        drop  = min(0.30, base_dropout + 0.03*expert_id)
        self.emb = nn.Parameter(torch.randn(16)*0.02)
        self.net = nn.Sequential(
            nn.Linear(in_dim+16, width), nn.ReLU(), nn.Dropout(drop),
            nn.Linear(width, max(32,width//2)), nn.ReLU(), nn.Dropout(drop),
        )
        out=max(32,width//2)
        self.mu=nn.Linear(out,1)
        self.logv=nn.Linear(out,1)
    def forward(self,z):
        B=z.size(0)
        e=self.emb.unsqueeze(0).expand(B,-1)
        h=self.net(torch.cat([z,e],dim=-1))
        mu=self.mu(h).squeeze(-1)
        logv=self.logv(h).squeeze(-1).clamp(-9,4)
        return mu, logv

class InterpretableMoE(nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.enc = SharedEncoder(n_features,cfg.enc_hidden,cfg.dropout)
        self.experts = nn.ModuleList([ExpertHead(cfg.enc_hidden,cfg.head_hidden,cfg.dropout,i) for i in range(cfg.n_experts)])
        self.gate = GatingNet(cfg.enc_hidden+9, cfg.n_experts, cfg.gate_dropout)

    def forward(self, x, *, use_topk, noise_std, temperature, train):
        z = self.enc(x)
        op = x[:,:, :3]
        gate_in = torch.cat([z, op[:,-1,:], op.mean(dim=1), op.std(dim=1)], dim=-1)

        logits = self.gate(gate_in)
        if train and noise_std>0:
            logits = logits + torch.randn_like(logits)*noise_std

        if use_topk:
            w, topk_idx = sparse_topk_softmax(logits, cfg.top_k, temperature)
            top1 = topk_idx[:,0]
        else:
            w = F.softmax(logits/max(1e-6,temperature), dim=-1)
            top1 = torch.argmax(w, dim=-1)

        mus=[]; vars_=[]
        for ex in self.experts:
            mu, logv = ex(z)
            mus.append(mu); vars_.append(torch.exp(logv))
        mus=torch.stack(mus,dim=-1)
        vars_=torch.stack(vars_,dim=-1)

        mean=torch.sum(w*mus,dim=-1)
        second=torch.sum(w*(vars_+mus**2),dim=-1)
        var=(second-mean**2).clamp_min(1e-6)
        log_var=torch.log(var)

        lb = switch_load_balance_loss(w, top1)
        ent = -(w*torch.log(w+1e-8)).sum(dim=-1).mean()
        dead = dead_expert_penalty(w, cfg.dead_floor)

        div=0.0
        E=mus.shape[-1]
        for i in range(E):
            for j in range(i+1,E):
                a=mus[:,i]-mus[:,i].mean()
                b=mus[:,j]-mus[:,j].mean()
                div += torch.abs(F.cosine_similarity(a.unsqueeze(-1), b.unsqueeze(-1), dim=-1)).mean()
        div = div / max(1,(E*(E-1)//2))

        return mean, log_var, w, lb, ent, div, dead

# ---------------------------
# Prepare domain
# ---------------------------
def prepare_domain(fd, scaler=None, kept_cols=None):
    tr, te, rul = load_cmapss_split(DATA_DIR, fd)
    tr=add_rul_train(tr,cfg.max_rul)
    te=add_rul_test(te,rul,cfg.max_rul)

    all_cols = base_feature_columns()
    if kept_cols is None:
        kept_cols = all_cols
        if cfg.drop_low_var:
            v = tr[all_cols].var(axis=0).values
            kept_cols = [c for c,vv in zip(all_cols,v) if vv>cfg.low_var_thresh]
            for c in ["op1","op2","op3"]:
                if c not in kept_cols:
                    kept_cols = ["op1","op2","op3"] + [x for x in kept_cols if x not in ["op1","op2","op3"]]

    X_tr_all, y_tr_all, u_tr_all, c_tr_all = make_windows(tr, cfg.window, cfg.stride, kept_cols)
    X_te_all, y_te_all, u_te_all, c_te_all = make_windows(te, cfg.window, cfg.stride, kept_cols)
    X_te_last, y_te_last, u_te_last, c_te_last = last_window_per_unit(X_te_all, y_te_all, u_te_all, c_te_all)

    if scaler is None:
        scaler = StandardScaler()
        scaler.fit(X_tr_all.reshape(-1, X_tr_all.shape[-1]))

    X_tr_all = scaler.transform(X_tr_all.reshape(-1,X_tr_all.shape[-1])).reshape(X_tr_all.shape)
    X_te_last = scaler.transform(X_te_last.reshape(-1,X_te_last.shape[-1])).reshape(X_te_last.shape)

    tr_idx, va_idx = split_by_units(u_tr_all, cfg.val_ratio_units, cfg.seed)
    X_tr, y_tr, u_tr, c_tr = X_tr_all[tr_idx], y_tr_all[tr_idx], u_tr_all[tr_idx], c_tr_all[tr_idx]
    X_va, y_va, u_va, c_va = X_tr_all[va_idx], y_tr_all[va_idx], u_tr_all[va_idx], c_tr_all[va_idx]

    y_te_raw = y_te_last.copy()

    if cfg.normalize_y:
        y_tr = y_tr/cfg.max_rul
        y_va = y_va/cfg.max_rul
        y_te = y_te_last/cfg.max_rul
    else:
        y_te = y_te_last

    return {
        "scaler": scaler, "kept_cols": kept_cols,
        "train": (X_tr,y_tr,u_tr,c_tr),
        "val": (X_va,y_va,u_va,c_va),
        "test_last": (X_te_last,y_te,u_te_last,c_te_last),
        "test_last_y_raw": y_te_raw
    }

# ---------------------------
# Interpretability
# ---------------------------
@torch.no_grad()
def gate_stats(model, X, n_show=1024):
    model.eval()
    xb=torch.tensor(X[:n_show],dtype=torch.float32).to(DEVICE)
    out = model(xb, use_topk=True, noise_std=0.0, temperature=1.0, train=False)
    w = out[2].cpu().numpy()
    print("Gate usage (avg weights):", np.round(w.mean(axis=0),4))

# ---------------------------
# Training
# ---------------------------
def train_on_domain(domain):
    X_tr,y_tr,u_tr,c_tr = domain["train"]
    X_va,y_va,u_va,c_va = domain["val"]

    model = InterpretableMoE(n_features=X_tr.shape[-1]).to(DEVICE)

    ds_tr = RULWindowDataset(X_tr,y_tr,u_tr,c_tr,domain_ids=np.zeros(len(X_tr),dtype=int))
    ds_va = RULWindowDataset(X_va,y_va,u_va,c_va,domain_ids=np.zeros(len(X_va),dtype=int))

    sampler = ContiguousEngineBatchSampler(ds_tr.unit_ids, ds_tr.cycles, cfg.batch_size, cfg.block_len, shuffle=True)
    dl_tr = DataLoader(ds_tr, batch_sampler=sampler)
    dl_va = DataLoader(ds_va, batch_size=cfg.batch_size, shuffle=False)

    opt = torch.optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=cfg.epochs)

    best_val=float("inf"); best_state=None; bad=0

    for ep in range(1,cfg.epochs+1):
        s = ramp(ep, cfg.warmup_epochs, cfg.ramp_epochs)
        use_topk = (ep > cfg.warmup_epochs)
        noise_std = lerp(cfg.gate_noise_max, cfg.gate_noise_min, s)
        temperature = lerp(cfg.temp_max, cfg.temp_min, s)

        mono_w   = cfg.lambda_mono*s
        smooth_w = cfg.lambda_smooth*s
        lb_w     = cfg.lambda_lb*s
        ent_w    = cfg.lambda_ent*s
        div_w    = cfg.lambda_div*s
        dead_w   = cfg.lambda_dead*s

        model.train()
        tr_loss=0.0
        for xb,yb,ub,cb,db in dl_tr:
            xb=xb.to(DEVICE); yb=yb.to(DEVICE)
            opt.zero_grad(set_to_none=True)

            mean, logv, w, lb, ent, div, dead = model(
                xb, use_topk=use_topk, noise_std=noise_std, temperature=temperature, train=True
            )

            nll = gaussian_nll(yb, mean, logv).mean()
            mse = F.mse_loss(mean, yb)
            hub = F.huber_loss(mean, yb, delta=cfg.huber_delta)
            mono, smooth = physics_losses(mean, ub, cb)

            if ep <= cfg.warmup_epochs:
                loss = 1.0*mse + 0.10*nll + cfg.huber_weight*hub
            else:
                loss = nll + cfg.aux_mse_weight*mse + cfg.huber_weight*hub

            loss = loss + mono_w*mono + smooth_w*smooth + lb_w*lb - ent_w*ent + div_w*div + dead_w*dead

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.grad_clip)
            opt.step()
            tr_loss += float(loss.detach().cpu())

        sched.step()

        # val loss + quick metrics
        model.eval()
        va_loss=0.0
        with torch.no_grad():
            for xb,yb,ub,cb,db in dl_va:
                xb=xb.to(DEVICE); yb=yb.to(DEVICE)
                mean, logv, *_ = model(xb, use_topk=True, noise_std=0.0, temperature=1.0, train=False)
                nll = gaussian_nll(yb, mean, logv).mean()
                mse = F.mse_loss(mean, yb)
                hub = F.huber_loss(mean, yb, delta=cfg.huber_delta)
                va_loss += float((nll + cfg.aux_mse_weight*mse + cfg.huber_weight*hub).cpu())
        va_loss /= max(1,len(dl_va))

        yhat_val, _ = predict_mc(model, X_va, n_mc=1)
        if cfg.normalize_y:
            yhat_val = yhat_val*cfg.max_rul
            y_val_raw = y_va*cfg.max_rul
        else:
            y_val_raw = y_va
        r2 = r2_score(y_val_raw, yhat_val)
        rmse = math.sqrt(mean_squared_error(y_val_raw, yhat_val))
        print(f"[Epoch {ep:02d}] s={s:.2f} topk={int(use_topk)} val={va_loss:.4f} | R2={r2:.4f} RMSE={rmse:.3f}")

        if va_loss + 1e-6 < best_val:
            best_val=va_loss
            best_state={k:v.detach().cpu().clone() for k,v in model.state_dict().items()}
            bad=0
        else:
            bad+=1
            if bad>=cfg.early_stop_patience:
                print(f"Early stopping (best val={best_val:.4f})")
                break

    if best_state is not None:
        model.load_state_dict(best_state)
    return model

# ---------------------------
# RUN
# ---------------------------
for fd in set(PRETRAIN_DOMAINS + TARGETS):
    for f in [f"train_{fd}.txt", f"test_{fd}.txt", f"RUL_{fd}.txt"]:
        ensure_exists(os.path.join(DATA_DIR, f))

domains={}
scaler=None
kept=None
for fd in PRETRAIN_DOMAINS:
    domains[fd]=prepare_domain(fd, scaler=scaler, kept_cols=kept)
    scaler=domains[fd]["scaler"]
    kept=domains[fd]["kept_cols"]
for fd in TARGETS:
    if fd not in domains:
        domains[fd]=prepare_domain(fd, scaler=scaler, kept_cols=kept)

models=[]
for mi in range(ENSEMBLE_SIZE):
    print("\n==============================")
    print(f" Training model {mi+1}/{ENSEMBLE_SIZE}")
    print("==============================")
    set_seed(cfg.seed + mi)
    models.append(train_on_domain(domains[PRETRAIN_DOMAINS[0]]))

for fd in TARGETS:
    print("\n==============================")
    print(f" Target {fd} | LAST window per unit")
    print("==============================")

    X_te, y_te, *_ = domains[fd]["test_last"]
    y_te_raw = domains[fd]["test_last_y_raw"]
    X_va, y_va, *_ = domains[fd]["val"]
    y_va_raw = (y_va*cfg.max_rul) if cfg.normalize_y else y_va

    mean, var = predict_mc(models[0], X_te, n_mc=cfg.mc_samples)
    if cfg.normalize_y:
        mean_raw = mean*cfg.max_rul
        var_raw  = var*(cfg.max_rul**2)
    else:
        mean_raw = mean; var_raw = var

    scale = calibrate_scale_min_width(models[0], X_va, y_va_raw, cfg.pi_alpha, cfg.mc_samples, max_over=0.01)
    var_raw = (scale**2)*var_raw

    r2 = r2_score(y_te_raw, mean_raw)
    rmse = math.sqrt(mean_squared_error(y_te_raw, mean_raw))
    mae = mean_absolute_error(y_te_raw, mean_raw)

    lo, hi = prediction_interval(mean_raw, var_raw, cfg.pi_alpha)
    cov = coverage(y_te_raw, lo, hi)

    print(f"Point: R2={r2:.4f} RMSE={rmse:.3f} MAE={mae:.3f}")
    print(f"UQ: {int((1-cfg.pi_alpha)*100)}% PI coverage={cov:.3f} | mean width={(hi-lo).mean():.3f} | cal_scale={scale:.3f}")

    gate_stats(models[0], X_te)

print("\n✅ Done.")



 Training model 1/1
[Epoch 01] s=0.00 topk=0 val=0.0894 | R2=0.6722 RMSE=23.885
[Epoch 02] s=0.00 topk=0 val=-0.3108 | R2=0.5821 RMSE=26.968
[Epoch 03] s=0.00 topk=0 val=-0.6049 | R2=0.6585 RMSE=24.381
[Epoch 04] s=0.00 topk=0 val=-0.9182 | R2=0.6446 RMSE=24.870
[Epoch 05] s=0.00 topk=0 val=-1.3225 | R2=0.6947 RMSE=23.051
[Epoch 06] s=0.00 topk=0 val=-0.8848 | R2=0.6259 RMSE=25.517
[Epoch 07] s=0.00 topk=0 val=-1.2841 | R2=0.7014 RMSE=22.798
[Epoch 08] s=0.00 topk=0 val=-1.0305 | R2=0.7097 RMSE=22.478
[Epoch 09] s=0.00 topk=0 val=-1.4801 | R2=0.7613 RMSE=20.381
[Epoch 10] s=0.00 topk=0 val=-1.5606 | R2=0.7627 RMSE=20.325
[Epoch 11] s=0.00 topk=0 val=-1.3519 | R2=0.7336 RMSE=21.531
[Epoch 12] s=0.00 topk=0 val=-1.1461 | R2=0.6607 RMSE=24.303
[Epoch 13] s=0.05 topk=1 val=-1.5128 | R2=0.8100 RMSE=18.187
[Epoch 14] s=0.10 topk=1 val=-1.5152 | R2=0.7922 RMSE=19.018
[Epoch 15] s=0.15 topk=1 val=-1.0954 | R2=0.6798 RMSE=23.606
[Epoch 16] s=0.20 topk=1 val=-1.1003 | R2=0.6635 RMSE=24.200
[Epo

FD004

In [1]:
# ============================================================
# FINAL Interpretable MOE for C-MAPSS RUL (ONE-CELL Colab)
# Fixes:
#  - Noisy Top-K gating + load balancing + entropy bonus (Sub-Obj 1)
#  - Physics constraints: monotonic + smooth (Sub-Obj 2)
#  - UQ: NLL + MC Dropout + PI calibration (Sub-Obj 3)
#  - Transfer-ready structure (Sub-Obj 4 hooks; single-domain run here)
# ============================================================

# ---------------------------
# USER SETTINGS
# ---------------------------
DATA_DIR = "/content"
PRETRAIN_DOMAINS = ["FD004"]
TARGETS = ["FD004"]
DO_FINETUNE = False
ENSEMBLE_SIZE = 1

# ---------------------------
# Install deps
# ---------------------------
import sys, subprocess, importlib
def _ensure(pkg, import_name=None):
    name = import_name or pkg
    try:
        importlib.import_module(name)
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "-q", "install", pkg])

_ensure("scikit-learn", "sklearn")
_ensure("scipy", "scipy")

# ---------------------------
# Imports
# ---------------------------
import os, math, random
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Sampler

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import norm

# ---------------------------
# Config
# ---------------------------
@dataclass
class CFG:
    window: int = 30
    stride: int = 1
    max_rul: int = 125
    normalize_y: bool = True
    val_ratio_units: float = 0.2
    seed: int = 42

    drop_low_var: bool = True
    low_var_thresh: float = 1e-6

    n_experts: int = 4
    top_k: int = 2
    enc_hidden: int = 192
    head_hidden: int = 192
    dropout: float = 0.08
    gate_dropout: float = 0.05

    gate_noise_max: float = 1.0
    gate_noise_min: float = 0.15
    temp_max: float = 2.0
    temp_min: float = 0.7

    epochs: int = 85
    batch_size: int = 128
    block_len: int = 12
    lr: float = 2e-3
    weight_decay: float = 1e-4
    grad_clip: float = 1.0
    early_stop_patience: int = 14

    aux_mse_weight: float = 0.55
    huber_weight: float = 0.15
    huber_delta: float = 0.08  # in normalized y scale

    lambda_mono: float = 0.10
    lambda_smooth: float = 0.03
    lambda_lb: float = 0.25
    lambda_ent: float = 0.02
    lambda_div: float = 0.01
    lambda_dead: float = 0.06
    dead_floor: float = 0.03

    warmup_epochs: int = 12
    ramp_epochs: int = 20

    mc_samples: int = 30
    pi_alpha: float = 0.10  # 90% PI

cfg = CFG()

# ---------------------------
# Utils
# ---------------------------
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_seed(cfg.seed)

def ensure_exists(path: str):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Missing file: {path}")

def base_feature_columns() -> List[str]:
    return ["op1", "op2", "op3"] + [f"s{i}" for i in range(1, 22)]

def ramp(ep, warmup, ramp_len):
    if ep <= warmup: return 0.0
    return float(min(1.0, (ep - warmup) / max(1, ramp_len)))

def lerp(a, b, t):
    return a + (b - a) * t

# ---------------------------
# C-MAPSS load
# ---------------------------
def load_cmapss_split(data_dir: str, fd: str):
    train_file = os.path.join(data_dir, f"train_{fd}.txt")
    test_file  = os.path.join(data_dir, f"test_{fd}.txt")
    rul_file   = os.path.join(data_dir, f"RUL_{fd}.txt")
    ensure_exists(train_file); ensure_exists(test_file); ensure_exists(rul_file)

    train_df = pd.read_csv(train_file, sep=r"\s+", header=None)
    test_df  = pd.read_csv(test_file,  sep=r"\s+", header=None)
    rul_df   = pd.read_csv(rul_file,   sep=r"\s+", header=None)

    cols = ["unit","cycle","op1","op2","op3"] + [f"s{i}" for i in range(1,22)]
    train_df.columns = cols
    test_df.columns  = cols
    rul_df.columns   = ["RUL_last"]
    return train_df, test_df, rul_df

def add_rul_train(df: pd.DataFrame, max_rul: int):
    df = df.copy()
    max_cycle = df.groupby("unit")["cycle"].max()
    df["RUL"] = df.apply(lambda r: max_cycle.loc[r["unit"]] - r["cycle"], axis=1)
    df["RUL"] = df["RUL"].clip(upper=max_rul)
    return df

def add_rul_test(test_df: pd.DataFrame, rul_df: pd.DataFrame, max_rul: int):
    df = test_df.copy()
    max_cycle = df.groupby("unit")["cycle"].max()
    rul_last = rul_df["RUL_last"].values
    mapping = {u: rul_last[u-1] for u in sorted(df["unit"].unique())}
    df["RUL"] = df.apply(lambda r: (max_cycle.loc[r["unit"]] + mapping[r["unit"]]) - r["cycle"], axis=1)
    df["RUL"] = df["RUL"].clip(upper=max_rul)
    return df

def make_windows(df, window, stride, feature_cols, target_col="RUL"):
    xs, ys, units, cycles = [], [], [], []
    for unit, g in df.groupby("unit"):
        g = g.sort_values("cycle")
        feats = g[feature_cols].values.astype(np.float32)
        targ  = g[target_col].values.astype(np.float32)
        cyc   = g["cycle"].values.astype(np.int32)

        for end in range(window-1, len(g), stride):
            start = end-window+1
            xs.append(feats[start:end+1])
            ys.append(targ[end])
            units.append(unit)
            cycles.append(cyc[end])
    return np.stack(xs), np.array(ys), np.array(units), np.array(cycles)

def last_window_per_unit(X, y, unit_ids, cycles):
    idx=[]
    for u in np.unique(unit_ids):
        m = unit_ids==u
        i = np.argmax(cycles[m])
        idx.append(np.where(m)[0][i])
    idx=np.array(idx)
    return X[idx], y[idx], unit_ids[idx], cycles[idx]

def split_by_units(unit_ids, val_ratio, seed):
    rng=np.random.default_rng(seed)
    units=np.unique(unit_ids)
    rng.shuffle(units)
    n_val=max(1,int(len(units)*val_ratio))
    val=set(units[:n_val].tolist())
    tr=np.array([i for i,u in enumerate(unit_ids) if u not in val])
    va=np.array([i for i,u in enumerate(unit_ids) if u in val])
    return tr, va

# ---------------------------
# Dataset + contiguous sampler
# ---------------------------
class RULWindowDataset(Dataset):
    def __init__(self, X, y, unit_ids, cycles, domain_ids):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.unit_ids = np.array(unit_ids)
        self.cycles = np.array(cycles)
        self.domain_ids = torch.tensor(domain_ids, dtype=torch.long)

    def __len__(self): return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], int(self.unit_ids[idx]), int(self.cycles[idx]), self.domain_ids[idx]

class ContiguousEngineBatchSampler(Sampler):
    def __init__(self, unit_ids, cycles, batch_size, block_len, shuffle=True):
        self.unit_ids=np.array(unit_ids)
        self.cycles=np.array(cycles)
        self.batch_size=batch_size
        self.block_len=block_len
        self.shuffle=shuffle
        self.units=np.unique(self.unit_ids)
        self.unit_to_sorted={}
        for u in self.units:
            idx=np.where(self.unit_ids==u)[0]
            idx=idx[np.argsort(self.cycles[idx])]
            self.unit_to_sorted[u]=idx
        self.engines_per_batch=max(1,batch_size//block_len)

    def __iter__(self):
        units=self.units.copy()
        if self.shuffle: np.random.shuffle(units)
        batch=[]
        for u in units:
            idx=self.unit_to_sorted[u]
            L=len(idx)
            if L<=self.block_len:
                take=np.random.choice(idx,size=self.block_len,replace=True)
            else:
                s=np.random.randint(0,L-self.block_len)
                take=idx[s:s+self.block_len]
            batch.extend(take.tolist())
            if len(batch)>=self.engines_per_batch*self.block_len:
                yield batch[:self.batch_size]
                batch=[]
        if batch: yield batch

    def __len__(self):
        return math.ceil(len(self.units)/self.engines_per_batch)

# ---------------------------
# Physics losses
# ---------------------------
def physics_losses(pred_mean, unit_ids, cycles, margin=0.0):
    unit_ids=np.array(unit_ids); cycles=np.array(cycles)
    mono_terms=[]; smooth_terms=[]
    for u in np.unique(unit_ids):
        idx=np.where(unit_ids==u)[0]
        if len(idx)<2: continue
        ord_idx=idx[np.argsort(cycles[idx])]
        p=pred_mean[ord_idx]
        mono=F.relu(p[1:]-p[:-1]+margin)
        mono_terms.append(mono.mean())
        if len(ord_idx)>=3:
            second=p[2:]-2*p[1:-1]+p[:-2]
            smooth_terms.append(torch.abs(second).mean())
    mono_loss=torch.stack(mono_terms).mean() if mono_terms else pred_mean.new_tensor(0.)
    smooth_loss=torch.stack(smooth_terms).mean() if smooth_terms else pred_mean.new_tensor(0.)
    return mono_loss, smooth_loss

# ---------------------------
# Gating + load balance
# ---------------------------
class GatingNet(nn.Module):
    def __init__(self, in_dim, n_experts, gate_dropout):
        super().__init__()
        h=max(64,in_dim//2)
        self.net=nn.Sequential(
            nn.Linear(in_dim,h),
            nn.ReLU(),
            nn.Dropout(gate_dropout),
            nn.Linear(h,n_experts),
        )
    def forward(self,x): return self.net(x)

def sparse_topk_softmax(logits, k, temperature):
    logits = logits / max(1e-6, temperature)
    topk_vals, topk_idx = torch.topk(logits, k=k, dim=-1)
    masked = torch.full_like(logits, float("-inf"))
    masked.scatter_(dim=-1, index=topk_idx, src=topk_vals)
    w = F.softmax(masked, dim=-1)
    return w, topk_idx

def switch_load_balance_loss(w, top1_idx):
    B,E=w.shape
    importance=w.sum(dim=0)/(B+1e-8)
    load=torch.bincount(top1_idx,minlength=E).float().to(w.device)/(B+1e-8)
    return E*torch.sum(importance*load)

def dead_expert_penalty(w, floor):
    avg = w.mean(dim=0)
    return F.relu(floor - avg).mean()

# ---------------------------
# UQ + calibration
# ---------------------------
def gaussian_nll(y, mean, log_var):
    return 0.5*(torch.exp(-log_var)*(y-mean)**2 + log_var)

def prediction_interval(mean, var, alpha):
    z = norm.ppf(1-alpha/2)
    std=np.sqrt(np.maximum(var,1e-8))
    return mean - z*std, mean + z*std

def coverage(y, lo, hi):
    return float(np.mean((y>=lo)&(y<=hi)))

# ✅ FIXED predict_mc: safe unpacking for 7-value forward()
@torch.no_grad()
def predict_mc(model, X, n_mc, batch_size=256):
    model.train()
    dl=DataLoader(torch.tensor(X,dtype=torch.float32),batch_size=batch_size,shuffle=False)
    means_all=[]; vars_all=[]
    for xb in dl:
        xb=xb.to(DEVICE)
        mc_m=[]; mc_v=[]
        for _ in range(n_mc):
            out = model(xb, use_topk=True, noise_std=0.0, temperature=1.0, train=True)
            mean, log_var = out[0], out[1]
            mc_m.append(mean)
            mc_v.append(torch.exp(log_var))
        mc_m=torch.stack(mc_m,dim=0)
        mc_v=torch.stack(mc_v,dim=0)
        mean_pred=mc_m.mean(dim=0)
        epistemic=mc_m.var(dim=0,unbiased=False)
        aleatoric=mc_v.mean(dim=0)
        total=aleatoric+epistemic
        means_all.append(mean_pred.cpu().numpy())
        vars_all.append(total.cpu().numpy())
    return np.concatenate(means_all), np.concatenate(vars_all)

def calibrate_scale_min_width(model, X_val, y_val_raw, alpha, n_mc, max_over=0.01):
    mean, var = predict_mc(model, X_val, n_mc=n_mc)
    if cfg.normalize_y:
        mean = mean * cfg.max_rul
        var  = var  * (cfg.max_rul**2)
    std = np.sqrt(np.maximum(var,1e-8))
    z = norm.ppf(1-alpha/2)
    target = 1-alpha
    scales = np.linspace(0.6, 3.0, 121)
    best=None
    for s in scales:
        lo = mean - z*s*std
        hi = mean + z*s*std
        cov = np.mean((y_val_raw>=lo)&(y_val_raw<=hi))
        if cov >= (target - max_over):
            best=s
            break
    return float(best if best is not None else scales[-1])

# ---------------------------
# Model
# ---------------------------
class SharedEncoder(nn.Module):
    def __init__(self, n_features, hidden, dropout):
        super().__init__()
        self.conv1=nn.Conv1d(n_features,96,3,padding=1)
        self.conv2=nn.Conv1d(96,96,3,padding=1)
        self.bn1=nn.BatchNorm1d(96)
        self.bn2=nn.BatchNorm1d(96)
        self.gru=nn.GRU(96,hidden,batch_first=True)
        self.drop=nn.Dropout(dropout)
    def forward(self,x):
        x=x.transpose(1,2)
        x=F.relu(self.bn1(self.conv1(x)))
        x=F.relu(self.bn2(self.conv2(x)))
        x=x.transpose(1,2)
        _,h=self.gru(x)
        return self.drop(h[-1])

class ExpertHead(nn.Module):
    def __init__(self, in_dim, hidden, base_dropout, expert_id):
        super().__init__()
        width = hidden if expert_id%2==0 else max(64,hidden//2)
        drop  = min(0.30, base_dropout + 0.03*expert_id)
        self.emb = nn.Parameter(torch.randn(16)*0.02)
        self.net = nn.Sequential(
            nn.Linear(in_dim+16, width), nn.ReLU(), nn.Dropout(drop),
            nn.Linear(width, max(32,width//2)), nn.ReLU(), nn.Dropout(drop),
        )
        out=max(32,width//2)
        self.mu=nn.Linear(out,1)
        self.logv=nn.Linear(out,1)
    def forward(self,z):
        B=z.size(0)
        e=self.emb.unsqueeze(0).expand(B,-1)
        h=self.net(torch.cat([z,e],dim=-1))
        mu=self.mu(h).squeeze(-1)
        logv=self.logv(h).squeeze(-1).clamp(-9,4)
        return mu, logv

class InterpretableMoE(nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.enc = SharedEncoder(n_features,cfg.enc_hidden,cfg.dropout)
        self.experts = nn.ModuleList([ExpertHead(cfg.enc_hidden,cfg.head_hidden,cfg.dropout,i) for i in range(cfg.n_experts)])
        self.gate = GatingNet(cfg.enc_hidden+9, cfg.n_experts, cfg.gate_dropout)

    def forward(self, x, *, use_topk, noise_std, temperature, train):
        z = self.enc(x)
        op = x[:,:, :3]
        gate_in = torch.cat([z, op[:,-1,:], op.mean(dim=1), op.std(dim=1)], dim=-1)

        logits = self.gate(gate_in)
        if train and noise_std>0:
            logits = logits + torch.randn_like(logits)*noise_std

        if use_topk:
            w, topk_idx = sparse_topk_softmax(logits, cfg.top_k, temperature)
            top1 = topk_idx[:,0]
        else:
            w = F.softmax(logits/max(1e-6,temperature), dim=-1)
            top1 = torch.argmax(w, dim=-1)

        mus=[]; vars_=[]
        for ex in self.experts:
            mu, logv = ex(z)
            mus.append(mu); vars_.append(torch.exp(logv))
        mus=torch.stack(mus,dim=-1)
        vars_=torch.stack(vars_,dim=-1)

        mean=torch.sum(w*mus,dim=-1)
        second=torch.sum(w*(vars_+mus**2),dim=-1)
        var=(second-mean**2).clamp_min(1e-6)
        log_var=torch.log(var)

        lb = switch_load_balance_loss(w, top1)
        ent = -(w*torch.log(w+1e-8)).sum(dim=-1).mean()
        dead = dead_expert_penalty(w, cfg.dead_floor)

        div=0.0
        E=mus.shape[-1]
        for i in range(E):
            for j in range(i+1,E):
                a=mus[:,i]-mus[:,i].mean()
                b=mus[:,j]-mus[:,j].mean()
                div += torch.abs(F.cosine_similarity(a.unsqueeze(-1), b.unsqueeze(-1), dim=-1)).mean()
        div = div / max(1,(E*(E-1)//2))

        return mean, log_var, w, lb, ent, div, dead

# ---------------------------
# Prepare domain
# ---------------------------
def prepare_domain(fd, scaler=None, kept_cols=None):
    tr, te, rul = load_cmapss_split(DATA_DIR, fd)
    tr=add_rul_train(tr,cfg.max_rul)
    te=add_rul_test(te,rul,cfg.max_rul)

    all_cols = base_feature_columns()
    if kept_cols is None:
        kept_cols = all_cols
        if cfg.drop_low_var:
            v = tr[all_cols].var(axis=0).values
            kept_cols = [c for c,vv in zip(all_cols,v) if vv>cfg.low_var_thresh]
            for c in ["op1","op2","op3"]:
                if c not in kept_cols:
                    kept_cols = ["op1","op2","op3"] + [x for x in kept_cols if x not in ["op1","op2","op3"]]

    X_tr_all, y_tr_all, u_tr_all, c_tr_all = make_windows(tr, cfg.window, cfg.stride, kept_cols)
    X_te_all, y_te_all, u_te_all, c_te_all = make_windows(te, cfg.window, cfg.stride, kept_cols)
    X_te_last, y_te_last, u_te_last, c_te_last = last_window_per_unit(X_te_all, y_te_all, u_te_all, c_te_all)

    if scaler is None:
        scaler = StandardScaler()
        scaler.fit(X_tr_all.reshape(-1, X_tr_all.shape[-1]))

    X_tr_all = scaler.transform(X_tr_all.reshape(-1,X_tr_all.shape[-1])).reshape(X_tr_all.shape)
    X_te_last = scaler.transform(X_te_last.reshape(-1,X_te_last.shape[-1])).reshape(X_te_last.shape)

    tr_idx, va_idx = split_by_units(u_tr_all, cfg.val_ratio_units, cfg.seed)
    X_tr, y_tr, u_tr, c_tr = X_tr_all[tr_idx], y_tr_all[tr_idx], u_tr_all[tr_idx], c_tr_all[tr_idx]
    X_va, y_va, u_va, c_va = X_tr_all[va_idx], y_tr_all[va_idx], u_tr_all[va_idx], c_tr_all[va_idx]

    y_te_raw = y_te_last.copy()

    if cfg.normalize_y:
        y_tr = y_tr/cfg.max_rul
        y_va = y_va/cfg.max_rul
        y_te = y_te_last/cfg.max_rul
    else:
        y_te = y_te_last

    return {
        "scaler": scaler, "kept_cols": kept_cols,
        "train": (X_tr,y_tr,u_tr,c_tr),
        "val": (X_va,y_va,u_va,c_va),
        "test_last": (X_te_last,y_te,u_te_last,c_te_last),
        "test_last_y_raw": y_te_raw
    }

# ---------------------------
# Interpretability
# ---------------------------
@torch.no_grad()
def gate_stats(model, X, n_show=1024):
    model.eval()
    xb=torch.tensor(X[:n_show],dtype=torch.float32).to(DEVICE)
    out = model(xb, use_topk=True, noise_std=0.0, temperature=1.0, train=False)
    w = out[2].cpu().numpy()
    print("Gate usage (avg weights):", np.round(w.mean(axis=0),4))

# ---------------------------
# Training
# ---------------------------
def train_on_domain(domain):
    X_tr,y_tr,u_tr,c_tr = domain["train"]
    X_va,y_va,u_va,c_va = domain["val"]

    model = InterpretableMoE(n_features=X_tr.shape[-1]).to(DEVICE)

    ds_tr = RULWindowDataset(X_tr,y_tr,u_tr,c_tr,domain_ids=np.zeros(len(X_tr),dtype=int))
    ds_va = RULWindowDataset(X_va,y_va,u_va,c_va,domain_ids=np.zeros(len(X_va),dtype=int))

    sampler = ContiguousEngineBatchSampler(ds_tr.unit_ids, ds_tr.cycles, cfg.batch_size, cfg.block_len, shuffle=True)
    dl_tr = DataLoader(ds_tr, batch_sampler=sampler)
    dl_va = DataLoader(ds_va, batch_size=cfg.batch_size, shuffle=False)

    opt = torch.optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=cfg.epochs)

    best_val=float("inf"); best_state=None; bad=0

    for ep in range(1,cfg.epochs+1):
        s = ramp(ep, cfg.warmup_epochs, cfg.ramp_epochs)
        use_topk = (ep > cfg.warmup_epochs)
        noise_std = lerp(cfg.gate_noise_max, cfg.gate_noise_min, s)
        temperature = lerp(cfg.temp_max, cfg.temp_min, s)

        mono_w   = cfg.lambda_mono*s
        smooth_w = cfg.lambda_smooth*s
        lb_w     = cfg.lambda_lb*s
        ent_w    = cfg.lambda_ent*s
        div_w    = cfg.lambda_div*s
        dead_w   = cfg.lambda_dead*s

        model.train()
        tr_loss=0.0
        for xb,yb,ub,cb,db in dl_tr:
            xb=xb.to(DEVICE); yb=yb.to(DEVICE)
            opt.zero_grad(set_to_none=True)

            mean, logv, w, lb, ent, div, dead = model(
                xb, use_topk=use_topk, noise_std=noise_std, temperature=temperature, train=True
            )

            nll = gaussian_nll(yb, mean, logv).mean()
            mse = F.mse_loss(mean, yb)
            hub = F.huber_loss(mean, yb, delta=cfg.huber_delta)
            mono, smooth = physics_losses(mean, ub, cb)

            if ep <= cfg.warmup_epochs:
                loss = 1.0*mse + 0.10*nll + cfg.huber_weight*hub
            else:
                loss = nll + cfg.aux_mse_weight*mse + cfg.huber_weight*hub

            loss = loss + mono_w*mono + smooth_w*smooth + lb_w*lb - ent_w*ent + div_w*div + dead_w*dead

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.grad_clip)
            opt.step()
            tr_loss += float(loss.detach().cpu())

        sched.step()

        # val loss + quick metrics
        model.eval()
        va_loss=0.0
        with torch.no_grad():
            for xb,yb,ub,cb,db in dl_va:
                xb=xb.to(DEVICE); yb=yb.to(DEVICE)
                mean, logv, *_ = model(xb, use_topk=True, noise_std=0.0, temperature=1.0, train=False)
                nll = gaussian_nll(yb, mean, logv).mean()
                mse = F.mse_loss(mean, yb)
                hub = F.huber_loss(mean, yb, delta=cfg.huber_delta)
                va_loss += float((nll + cfg.aux_mse_weight*mse + cfg.huber_weight*hub).cpu())
        va_loss /= max(1,len(dl_va))

        yhat_val, _ = predict_mc(model, X_va, n_mc=1)
        if cfg.normalize_y:
            yhat_val = yhat_val*cfg.max_rul
            y_val_raw = y_va*cfg.max_rul
        else:
            y_val_raw = y_va
        r2 = r2_score(y_val_raw, yhat_val)
        rmse = math.sqrt(mean_squared_error(y_val_raw, yhat_val))
        print(f"[Epoch {ep:02d}] s={s:.2f} topk={int(use_topk)} val={va_loss:.4f} | R2={r2:.4f} RMSE={rmse:.3f}")

        if va_loss + 1e-6 < best_val:
            best_val=va_loss
            best_state={k:v.detach().cpu().clone() for k,v in model.state_dict().items()}
            bad=0
        else:
            bad+=1
            if bad>=cfg.early_stop_patience:
                print(f"Early stopping (best val={best_val:.4f})")
                break

    if best_state is not None:
        model.load_state_dict(best_state)
    return model

# ---------------------------
# RUN
# ---------------------------
for fd in set(PRETRAIN_DOMAINS + TARGETS):
    for f in [f"train_{fd}.txt", f"test_{fd}.txt", f"RUL_{fd}.txt"]:
        ensure_exists(os.path.join(DATA_DIR, f))

domains={}
scaler=None
kept=None
for fd in PRETRAIN_DOMAINS:
    domains[fd]=prepare_domain(fd, scaler=scaler, kept_cols=kept)
    scaler=domains[fd]["scaler"]
    kept=domains[fd]["kept_cols"]
for fd in TARGETS:
    if fd not in domains:
        domains[fd]=prepare_domain(fd, scaler=scaler, kept_cols=kept)

models=[]
for mi in range(ENSEMBLE_SIZE):
    print("\n==============================")
    print(f" Training model {mi+1}/{ENSEMBLE_SIZE}")
    print("==============================")
    set_seed(cfg.seed + mi)
    models.append(train_on_domain(domains[PRETRAIN_DOMAINS[0]]))

for fd in TARGETS:
    print("\n==============================")
    print(f" Target {fd} | LAST window per unit")
    print("==============================")

    X_te, y_te, *_ = domains[fd]["test_last"]
    y_te_raw = domains[fd]["test_last_y_raw"]
    X_va, y_va, *_ = domains[fd]["val"]
    y_va_raw = (y_va*cfg.max_rul) if cfg.normalize_y else y_va

    mean, var = predict_mc(models[0], X_te, n_mc=cfg.mc_samples)
    if cfg.normalize_y:
        mean_raw = mean*cfg.max_rul
        var_raw  = var*(cfg.max_rul**2)
    else:
        mean_raw = mean; var_raw = var

    scale = calibrate_scale_min_width(models[0], X_va, y_va_raw, cfg.pi_alpha, cfg.mc_samples, max_over=0.01)
    var_raw = (scale**2)*var_raw

    r2 = r2_score(y_te_raw, mean_raw)
    rmse = math.sqrt(mean_squared_error(y_te_raw, mean_raw))
    mae = mean_absolute_error(y_te_raw, mean_raw)

    lo, hi = prediction_interval(mean_raw, var_raw, cfg.pi_alpha)
    cov = coverage(y_te_raw, lo, hi)

    print(f"Point: R2={r2:.4f} RMSE={rmse:.3f} MAE={mae:.3f}")
    print(f"UQ: {int((1-cfg.pi_alpha)*100)}% PI coverage={cov:.3f} | mean width={(hi-lo).mean():.3f} | cal_scale={scale:.3f}")

    gate_stats(models[0], X_te)

print("\n✅ Done.")



 Training model 1/1
[Epoch 01] s=0.00 topk=0 val=-0.1370 | R2=-0.0879 RMSE=43.112
[Epoch 02] s=0.00 topk=0 val=-0.4181 | R2=-0.0584 RMSE=42.523
[Epoch 03] s=0.00 topk=0 val=-0.6923 | R2=0.5067 RMSE=29.030
[Epoch 04] s=0.00 topk=0 val=-0.7007 | R2=0.6014 RMSE=26.095
[Epoch 05] s=0.00 topk=0 val=-0.7130 | R2=0.3685 RMSE=32.848
[Epoch 06] s=0.00 topk=0 val=-0.6488 | R2=0.3043 RMSE=34.476
[Epoch 07] s=0.00 topk=0 val=-1.0896 | R2=0.5807 RMSE=26.764
[Epoch 08] s=0.00 topk=0 val=-1.0222 | R2=0.5436 RMSE=27.926
[Epoch 09] s=0.00 topk=0 val=-0.6560 | R2=0.1965 RMSE=37.052
[Epoch 10] s=0.00 topk=0 val=-0.9359 | R2=0.4450 RMSE=30.793
[Epoch 11] s=0.00 topk=0 val=-0.4969 | R2=0.2563 RMSE=35.645
[Epoch 12] s=0.00 topk=0 val=-1.0868 | R2=0.5786 RMSE=26.831
[Epoch 13] s=0.05 topk=1 val=-0.9888 | R2=0.4901 RMSE=29.515
[Epoch 14] s=0.10 topk=1 val=-1.0477 | R2=0.5623 RMSE=27.345
[Epoch 15] s=0.15 topk=1 val=-1.0026 | R2=0.5436 RMSE=27.926
[Epoch 16] s=0.20 topk=1 val=-1.1605 | R2=0.6136 RMSE=25.694
[

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive
